# Project 2 - Ames Housing Data Price Prediction Model (part 1)

## Data cleaning

### Contents:
- [Data Cleaning of train set](#Data-Cleaning---train-set)
- [Data Cleaning of test set](#Data-Cleaning---test-set)
- [Combining test and train sets](#Combining-both-test-and-train)
- [Encoding of categorical data](#Encoding-of-categorical-data)
- [Splitting back to train and test sets](#Split-back-into-train-and-test-set)

In [204]:
#Import neccessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Data Cleaning - train set

In [205]:
#set max columns to see all columns
pd.set_option('display.max_columns', 500)

In [206]:
#read in dataset
df = pd.read_csv('./datasets/train.csv')

In [207]:
df.shape

(2051, 81)

In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
Id                 2051 non-null int64
PID                2051 non-null int64
MS SubClass        2051 non-null int64
MS Zoning          2051 non-null object
Lot Frontage       1721 non-null float64
Lot Area           2051 non-null int64
Street             2051 non-null object
Alley              140 non-null object
Lot Shape          2051 non-null object
Land Contour       2051 non-null object
Utilities          2051 non-null object
Lot Config         2051 non-null object
Land Slope         2051 non-null object
Neighborhood       2051 non-null object
Condition 1        2051 non-null object
Condition 2        2051 non-null object
Bldg Type          2051 non-null object
House Style        2051 non-null object
Overall Qual       2051 non-null int64
Overall Cond       2051 non-null int64
Year Built         2051 non-null int64
Year Remod/Add     2051 non-null int64
Roof Style         20

In [209]:
num_col = [col for col in df.columns if (df[col].dtype == int or df[col].dtype == float)]
num_col

['Id',
 'PID',
 'MS SubClass',
 'Lot Frontage',
 'Lot Area',
 'Overall Qual',
 'Overall Cond',
 'Year Built',
 'Year Remod/Add',
 'Mas Vnr Area',
 'BsmtFin SF 1',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 '1st Flr SF',
 '2nd Flr SF',
 'Low Qual Fin SF',
 'Gr Liv Area',
 'Bsmt Full Bath',
 'Bsmt Half Bath',
 'Full Bath',
 'Half Bath',
 'Bedroom AbvGr',
 'Kitchen AbvGr',
 'TotRms AbvGrd',
 'Fireplaces',
 'Garage Yr Blt',
 'Garage Cars',
 'Garage Area',
 'Wood Deck SF',
 'Open Porch SF',
 'Enclosed Porch',
 '3Ssn Porch',
 'Screen Porch',
 'Pool Area',
 'Misc Val',
 'Mo Sold',
 'Yr Sold',
 'SalePrice']

In [210]:
#ensure that the Id and PID are unique
print('total unique rows of Id:', len(df['Id'].unique()))
print('total unique rows of PID:', len(df['PID'].unique()))

total unique rows of Id: 2051
total unique rows of PID: 2051


In [211]:
df.columns

Index(['Id', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
       'G

In [212]:
#breaking down into sub column categories for easy reference
class_col = ['Id', 'PID', 'MS SubClass', 'MS Zoning', 'Bldg Type', 'House Style', 'Overall Qual', 'Overall Cond',
             'Lot Shape',
             'Land Contour', 'Lot Config', 'Land Slope', 'Functional']

date_col = ['Id', 'PID', 'Year Built', 'Year Remod/Add', 'Mo Sold', 'Yr Sold']

ext_col = ['Id', 'PID', 'Street', 'Alley', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type','Mas Vnr Area', 'Exter Qual',
           'Exter Cond', 'Foundation', 'Neighborhood','Condition 1', 'Condition 2', 'Paved Drive', 'Pool QC','Fence']

int_col = ['Id', 'PID', 'Bsmt Full Bath','Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr','Kitchen AbvGr', 
           'Kitchen Qual', 'TotRms AbvGrd','Bsmt Qual','Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 
           'BsmtFin SF 1','BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 'Fireplaces', 
           'Fireplace Qu', 'Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Cars', 'Garage Area', 
           'Garage Qual','Garage Cond']
bsmt_col = ['Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1','BsmtFin Type 2', 'BsmtFin SF 2', 
            'Bsmt Unf SF','Total Bsmt SF', 'Bsmt Full Bath','Bsmt Half Bath']

util_col = ['Id', 'PID', 'Utilities', 'Heating', 'Heating QC', 'Central Air', 'Electrical', 'Misc Feature', 'Misc Val']

sqft_col = ['Id', 'PID', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Lot Area', 'Wood Deck SF', 'Open Porch SF', 
            'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area']

bath_room_col = ['Id', 'PID', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr',
                 'Kitchen Qual', 'TotRms AbvGrd']

sale_col = ['Id', 'PID', 'Mo Sold', 'Yr Sold', 'Sale Type', 'SalePrice']

qc_col = ['Id', 'PID', 'Overall Qual', 'Overall Cond', 'Exter Qual', 'Exter Cond', 'Pool QC', 'Kitchen Qual', 'Bsmt Qual',
          'Bsmt Cond', 'Fireplace Qu', 'Garage Qual','Garage Cond', 'Heating QC']

gar_col = ['Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Cars', 'Garage Area', 
           'Garage Qual','Garage Cond']

In [213]:
#to change it back to list again
#pd.read_csv('datasets/lol.csv').squeeze().to_list()

### Null values - Numerical columns

We shall deal with the __numerical__ Null values first:

In [214]:
#get null count
df_null_num = pd.DataFrame(df[num_col].isnull().sum(), columns=['null_count'])

In [215]:
df_null_num = df_null_num[df_null_num['null_count'] > 0].sort_values('null_count', ascending=False).sort_index()
df_null_num

,null_count
Bsmt Full Bath,2
Bsmt Half Bath,2
Bsmt Unf SF,1
BsmtFin SF 1,1
BsmtFin SF 2,1
Garage Area,1
Garage Cars,1
Garage Yr Blt,114
Lot Frontage,330
Mas Vnr Area,22


__Handling of `Bsmt Full Bath` column__

In [216]:
df[df['Bsmt Full Bath'].isna()]['Bsmt Full Bath']

616    NaN
1327   NaN
Name: Bsmt Full Bath, dtype: float64

In [217]:
df.loc[df[df['Bsmt Full Bath'].isna()].index, 'Bsmt Full Bath'] = 0.0

In [218]:
#confirm that the NA has been changed
df.groupby('Bsmt Full Bath').count()['Id'].sum()

2051

__Handling of `Bsmt Half Bath` column__

In [219]:
df[df['Bsmt Half Bath'].isna()]['Bsmt Half Bath']

616    NaN
1327   NaN
Name: Bsmt Half Bath, dtype: float64

In [220]:
df.loc[df[df['Bsmt Half Bath'].isna()].index, 'Bsmt Half Bath'] = 0.0

In [221]:
#confirm that the NA has been changed
df.groupby('Bsmt Half Bath').count()['Id']

Bsmt Half Bath
0.0    1925
1.0     122
2.0       4
Name: Id, dtype: int64

__Handling of `Bsmt Unf SF`, `BsmtFin SF 1`, `BsmtFin SF 2` columns__

In [222]:
df.loc[df[df['Bsmt Unf SF'].isna()].index, 'Bsmt Unf SF'] = 0

In [223]:
df.groupby('Bsmt Unf SF')['Id'].count().sum()

2051

In [224]:
df.loc[df[df['BsmtFin SF 1'].isna()].index, 'BsmtFin SF 1'] = 0

In [225]:
df.groupby('BsmtFin SF 1')['Id'].count().sum()

2051

In [226]:
df.loc[df[df['BsmtFin SF 2'].isna()].index, 'BsmtFin SF 2'] = 0

In [227]:
df.groupby('BsmtFin SF 2')['Id'].count().sum()

2051

__Handling of `Garage Area`, `Garage Cars` columns__

In [228]:
df[df['Garage Area'] == 0].head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
28,2243,911103060,190,C (all),75.0,8250,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Feedr,Norm,2fmCon,2Story,5,6,1895,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,957.0,957.0,GasA,Fa,N,SBrkr,1034,957,0,1991,0.0,0.0,2,0,4,2,TA,9,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,N,0,0,133,0,0,0,NaN,NaN,NaN,0,6,2007,WD,119600
53,330,923226250,160,RM,21.0,1476,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,GLQ,176.0,Unf,0.0,370.0,546.0,GasA,Ex,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,200,26,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,76000
65,2278,923202134,20,RL,70.0,8402,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Feedr,Norm,1Fam,1Story,5,5,2007,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,ALQ,206.0,Unf,0.0,914.0,1120.0,GasA,Ex,Y,SBrkr,1120,0,0,1120,0.0,0.0,1,0,3,1,TA,6,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,30,0,0,0,0,NaN,NaN,NaN,0,12,2007,New,147000
79,2235,910201050,50,RM,50.0,7288,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Artery,Norm,1Fam,1.5Fin,5,7,1925,2003,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,Gd,BrkTil,TA,Po,No,Unf,0.0,Unf,0.0,936.0,936.0,GasA,Ex,Y,SBrkr,936,665,0,1601,0.0,0.0,2,0,3,1,TA,6,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,N,0,0,0,176,0,0,NaN,NaN,NaN,0,9,2007,WD,129850
101,2084,905476170,30,RL,58.0,9098,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,4,7,1920,2002,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,BrkTil,TA,TA,Mn,ALQ,348.0,Unf,0.0,180.0,528.0,GasA,Ex,Y,SBrkr,605,0,0,605,1.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,N,0,0,144,0,0,0,NaN,NaN,NaN,0,7,2007,WD,86000


In [229]:
#Find out where the null values lie at 
print(df[df['Garage Area'].isna()].index)
print(df[df['Garage Cars'].isna()].index)

Int64Index([1712], dtype='int64')
Int64Index([1712], dtype='int64')


In [230]:
#change them to 0 as they are the odd one out
df.at[1712, 'Garage Area'] = 0
df.at[1712, 'Garage Cars'] = 0

__Handling of `Mas Vnr Area` column__

In [231]:
df['Mas Vnr Area'].shape

(2051,)

In [232]:
df[df['Mas Vnr Area'].isna()][ext_col].shape

(22, 17)

In [233]:
df[df['Mas Vnr Area'] == 0].shape

(1216, 81)

In [234]:
#assign Mas Vnr Area NA values to 0
df.loc[df[df['Mas Vnr Area'].isna()].index, ['Mas Vnr Area']] = 0

In [235]:
df[df['Mas Vnr Area'] == 0].shape

(1238, 81)

In [236]:
df[df['Mas Vnr Area'].isna()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice


__Handling of `Garage Yr Blt` column__

In [237]:
len(df[df['Garage Yr Blt'].isna()].index)

114

Since there are only a few rows that has null values out of the whole dataset, it does not make sense to impute the null values by giving a mean year per neighbourhood, as it might not be accurate and will not make sense if the mean year turns out to be less than the year that the house was being built. 

We can directly drop it. 

In [238]:
df.drop(index=df[df['Garage Yr Blt'].isna()].index, inplace=True)

In [239]:
#dropping 114 rows which were null
df.shape

(1937, 81)

__Handling of `Lot Frontage` column__

In [240]:
df[df['Lot Frontage'].isna()].shape

(324, 81)

In [241]:
#assign NA values to 0
df.loc[df[df['Lot Frontage'].isna()].index, ['Lot Frontage']] = 0

In [242]:
df[df['Lot Frontage'].isna()].shape

(0, 81)

__Handling of `Total Bsmt SF` column__

In [243]:
df[df['Total Bsmt SF'].isna()][ext_col].shape

(1, 17)

In [244]:
#assign Total Bsmt SF NA values to 0
df.loc[df[df['Total Bsmt SF'].isna()].index, ['Total Bsmt SF']] = 0

In [245]:
df[df['Total Bsmt SF'].isna()][ext_col].shape

(0, 17)

In [246]:
#to confirm if there are no more null values in numerical columns
df_null_num = pd.DataFrame(df[num_col].isnull().sum(), columns=['null_count']).sort_values(by='null_count',ascending=False)
df_null_num = df_null_num[df_null_num['null_count'] > 0].sort_values('null_count', ascending=False).sort_index()
df_null_num

,null_count


In [247]:
#final df shape
df.shape

(1937, 81)

### Null values - Categorical columns

In [248]:
#get null values of categorical columns
df_null_cat = pd.DataFrame(df.isnull().sum(), columns=['null_count']).sort_values(by='null_count',ascending=False)
df_null_cat = df_null_cat[df_null_cat['null_count'] > 0].sort_values('null_count', ascending=False).sort_index()

df_null_cat

,null_count
Alley,1813
Bsmt Cond,46
Bsmt Exposure,49
Bsmt Qual,46
BsmtFin Type 1,46
BsmtFin Type 2,47
Fence,1558
Fireplace Qu,895
Mas Vnr Type,21
Misc Feature,1875


__Handling of `Alley` column__

In [249]:
#total non-null values for Alley:
df['Alley'].value_counts()

Grvl    71
Pave    53
Name: Alley, dtype: int64

In [250]:
#total null values for Alley:
df['Alley'].isnull().sum()

1813

In [251]:
#Percentage of null values in Alley
df['Alley'].isnull().sum()/2051

0.8839590443686007

Since there are too many null values (almost 88% of the total observations) in `Alley`, and unlikely will influence the house price, we shall drop it.   

In [252]:
df.drop(columns='Alley', inplace=True)

In [253]:
df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,0.0,13517,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,NaN,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,SBrkr,744,700,0,1444,0.0,0.0,2,1,3,1,TA,7,Typ,0,NaN,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA,TA,Y,SBrkr,831,614,0,1445,0.0,0.0,2,0,3,1,TA,6,Typ,0,NaN,Detchd,1957.0,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


__Handling of `Bsmt Cond` column__

In [254]:
#total non-null values
df['Bsmt Cond'].value_counts().sum()

1891

In [255]:
#total null values
df['Bsmt Cond'].isnull().sum()

46

In [256]:
df['Bsmt Cond'].value_counts()

TA    1749
Gd      84
Fa      53
Ex       3
Po       2
Name: Bsmt Cond, dtype: int64

In [257]:
df[df['Bsmt Cond'].isnull()].head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
12,807,906226060,70,RL,120.0,26400,Pave,Reg,Bnk,AllPub,FR2,Gtl,SawyerW,Feedr,Norm,1Fam,2Story,5,7,1880,2007,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,GasA,Ex,Y,SBrkr,1288,728,0,2016,0.0,0.0,1,0,4,1,TA,7,Mod,1,TA,Attchd,1900.0,Unf,2.0,576.0,TA,TA,P,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2009,WD,131000
93,811,906226100,90,RL,64.0,7018,Pave,Reg,Bnk,AllPub,Inside,Gtl,SawyerW,Norm,Norm,Duplex,1Story,5,5,1979,1979,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,GasA,TA,Y,SBrkr,1535,0,0,1535,0.0,0.0,2,0,4,2,TA,8,Typ,0,NaN,Attchd,1979.0,Unf,2.0,400.0,TA,TA,Y,0,0,0,0,0,0,NaN,GdPrv,NaN,0,6,2009,WD,118858
146,888,908128060,85,RL,64.0,7301,Pave,Reg,Lvl,AllPub,Corner,Gtl,Edwards,Norm,Norm,1Fam,SFoyer,7,5,2003,2003,Gable,CompShg,HdBoard,HdBoard,BrkFace,500.0,Gd,TA,Slab,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,GasA,Ex,Y,SBrkr,495,1427,0,1922,0.0,0.0,3,0,4,1,Gd,7,Typ,1,Ex,BuiltIn,2003.0,RFn,2.0,672.0,TA,TA,Y,0,0,177,0,0,0,NaN,NaN,NaN,0,7,2009,ConLD,198500
183,1554,910251050,20,A (agr),80.0,14584,Pave,Reg,Low,AllPub,Inside,Mod,IDOTRR,Norm,Norm,1Fam,1Story,1,5,1952,1952,Gable,CompShg,AsbShng,VinylSd,None,0.0,Fa,Po,Slab,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,Wall,Po,N,FuseA,733,0,0,733,0.0,0.0,1,0,2,1,Fa,4,Sal,0,NaN,Attchd,1952.0,Unf,2.0,487.0,Fa,Po,N,0,0,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,13100
240,2740,905451050,20,RL,80.0,12048,Pave,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,5,6,1952,2002,Gable,CompShg,Wd Sdng,Wd Sdng,BrkFace,232.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,GasA,Gd,Y,SBrkr,1488,0,0,1488,0.0,0.0,1,0,3,1,TA,7,Typ,1,Ex,Attchd,2002.0,RFn,2.0,569.0,TA,TA,Y,0,189,36,0,348,0,NaN,NaN,NaN,0,4,2006,WD,135000


In [258]:
df.loc[df[df['Bsmt Cond'].isnull()].index, 'Bsmt Cond'] = 'None'

In [259]:
#total null values
print(df['Bsmt Cond'].isnull().sum())

#ensure that the NA values are in the value counts
df['Bsmt Cond'].value_counts()

0


TA      1749
Gd        84
Fa        53
None      46
Ex         3
Po         2
Name: Bsmt Cond, dtype: int64

__Handling of `Bsmt Exposure` column__

In [260]:
#total non-null values
df['Bsmt Exposure'].value_counts().sum()

1888

In [261]:
#total null values
df['Bsmt Exposure'].isnull().sum()

49

In [262]:
df['Bsmt Exposure'].value_counts()

No    1259
Av     276
Gd     195
Mn     158
Name: Bsmt Exposure, dtype: int64

In [263]:
df.loc[df[df['Bsmt Exposure'].isnull()].index, 'Bsmt Exposure'] = 'None'

In [264]:
#total null values
print(df['Bsmt Cond'].isnull().sum())

#ensure that the NA values are in the value counts
df['Bsmt Exposure'].value_counts()

0


No      1259
Av       276
Gd       195
Mn       158
None      49
Name: Bsmt Exposure, dtype: int64

__Handling of `Bsmt Qual` column__

In [265]:
#total non-null values
df['Bsmt Qual'].value_counts().sum()

1891

In [266]:
#total null values
df['Bsmt Qual'].isnull().sum()

46

In [267]:
df['Bsmt Qual'].value_counts()

Gd    839
TA    817
Ex    184
Fa     50
Po      1
Name: Bsmt Qual, dtype: int64

In [268]:
df.loc[df[df['Bsmt Qual'].isnull()].index, 'Bsmt Qual'] = 'None'

In [269]:
#total null values
print(df['Bsmt Qual'].isnull().sum())

#ensure that the NA values are in the value counts
df['Bsmt Qual'].value_counts()

0


Gd      839
TA      817
Ex      184
Fa       50
None     46
Po        1
Name: Bsmt Qual, dtype: int64

__Handling of `BsmtFin Type 1` column__

In [270]:
#total non-null values
df['BsmtFin Type 1'].value_counts().sum()

1891

In [271]:
#total null values
df['BsmtFin Type 1'].isnull().sum()

46

In [272]:
df['BsmtFin Type 1'].value_counts()

GLQ    599
Unf    544
ALQ    279
BLQ    194
Rec    177
LwQ     98
Name: BsmtFin Type 1, dtype: int64

In [273]:
df.loc[df[df['BsmtFin Type 1'].isnull()].index, 'BsmtFin Type 1'] = 'None'

In [274]:
#total null values
print(df['BsmtFin Type 1'].isnull().sum())

#ensure that the NA values are in the value counts
df['BsmtFin Type 1'].value_counts()

0


GLQ     599
Unf     544
ALQ     279
BLQ     194
Rec     177
LwQ      98
None     46
Name: BsmtFin Type 1, dtype: int64

__Handling of `BsmtFin Type 2` column__

In [275]:
#total non-null values
df['BsmtFin Type 2'].value_counts().sum()

1890

In [276]:
#total null values
df['BsmtFin Type 2'].isnull().sum()

47

In [277]:
df['BsmtFin Type 2'].value_counts()

Unf    1648
Rec      79
LwQ      60
BLQ      47
ALQ      34
GLQ      22
Name: BsmtFin Type 2, dtype: int64

In [278]:
df.loc[df[df['BsmtFin Type 2'].isnull()].index, 'BsmtFin Type 2'] = 'None'

In [279]:
#total null values
print(df['BsmtFin Type 2'].isnull().sum())

#ensure that the NA values are in the value counts
df['BsmtFin Type 2'].value_counts()

0


Unf     1648
Rec       79
LwQ       60
None      47
BLQ       47
ALQ       34
GLQ       22
Name: BsmtFin Type 2, dtype: int64

__Handling of `Fence` column__

In [280]:
#total non-null values
df['Fence'].value_counts().sum()

379

In [281]:
#total null values
df['Fence'].isnull().sum()

1558

In [282]:
df['Fence'].value_counts()

MnPrv    214
GdPrv     82
GdWo      73
MnWw      10
Name: Fence, dtype: int64

In [283]:
ext_col = ['Id', 'PID', 'Street', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Mas Vnr Area', 'Exter Qual', 
           'Exter Cond', 'Foundation', 'Neighborhood', 'Condition 1', 'Condition 2', 'Paved Drive', 'Pool QC', 
           'Fence']
df[ext_col].head()

,Id,PID,Street,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Neighborhood,Condition 1,Condition 2,Paved Drive,Pool QC,Fence
0,109,533352170,Pave,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,Sawyer,RRAe,Norm,Y,NaN,NaN
1,544,531379050,Pave,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,SawyerW,Norm,Norm,Y,NaN,NaN
2,153,535304180,Pave,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,NAmes,Norm,Norm,Y,NaN,NaN
3,318,916386060,Pave,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Timber,Norm,Norm,Y,NaN,NaN
4,255,906425045,Pave,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,SawyerW,Norm,Norm,N,NaN,NaN


Even though `Fence` column has many NA values, it could be one of the good predictors of price, so we are keeping it by assigning another neutral category for ordinal encoding later on. 

In [284]:
df.loc[df[df['Fence'].isnull()].index, 'Fence'] = 'None'

In [286]:
#total null values
print(df['Fence'].isnull().sum())

#ensure that the NA values are in the value counts
df['Fence'].value_counts()

0


None     1558
MnPrv     214
GdPrv      82
GdWo       73
MnWw       10
Name: Fence, dtype: int64

__Handling of `Fireplace Qu` column__

In [287]:
#total non-null values
df['Fireplace Qu'].value_counts().sum()

1042

In [288]:
#total null values
df['Fireplace Qu'].isnull().sum()

895

In [289]:
df['Fireplace Qu'].value_counts()

Gd    517
TA    405
Fa     59
Ex     31
Po     30
Name: Fireplace Qu, dtype: int64

In [290]:
df[df['Fireplace Qu'].isna() & (df["Fireplaces"] == 0)].shape

(895, 80)

Similar to `Fence` column, `Fireplace Qu` could be one of the good predictors of price. Furthermore, since `Fireplaces = 0` also equal to each other, we are keeping it by assigning another neutral category for ordinal encoding later on. 

In [291]:
df.loc[df[df['Fireplace Qu'].isnull()].index, 'Fireplace Qu'] = 'None'

In [292]:
#total null values
print(df['Fireplace Qu'].isnull().sum())

#ensure that the NA values are in the value counts
df['Fireplace Qu'].value_counts()

0


None    895
Gd      517
TA      405
Fa       59
Ex       31
Po       30
Name: Fireplace Qu, dtype: int64

__Handling of `Mas Vnr Type` column__

In [293]:
#total non-null values
df['Mas Vnr Type'].value_counts().sum()

1916

In [294]:
#total null values
df['Mas Vnr Type'].isnull().sum()

21

In [295]:
df['Mas Vnr Type'].value_counts()

None       1116
BrkFace     620
Stone       168
BrkCmn       12
Name: Mas Vnr Type, dtype: int64

In [296]:
df[df['Mas Vnr Type'].isna() & (df['Mas Vnr Area'] == 0)].shape

(21, 80)

`Mas Vnr Type` could be one of the good predictors of price. Furthermore, since `Mas Vnr Area = 0` also equal to each other and both in sync, we are going to assign it as `'None'` as the existing category.

In [297]:
df.loc[df[df['Mas Vnr Type'].isnull()].index, 'Mas Vnr Type'] = 'None'

In [298]:
#total null values
print(df['Mas Vnr Type'].isnull().sum())

#ensure that the NA values are in the value counts
df['Mas Vnr Type'].value_counts()

0


None       1137
BrkFace     620
Stone       168
BrkCmn       12
Name: Mas Vnr Type, dtype: int64

__Handling of `Misc Feature` column__

In [299]:
#total non-null values
df['Misc Feature'].value_counts().sum()

62

In [300]:
#total null values
df['Misc Feature'].isnull().sum()

1875

In [301]:
df['Misc Feature'].value_counts()

Shed    54
Gar2     4
Othr     2
Elev     1
TenC     1
Name: Misc Feature, dtype: int64

In [302]:
df[df['Misc Feature'].isna()].shape

(1875, 80)

`Misc Feature` could be one of the good predictors of price, we never know. Hence, we are going to assign it a neutral value as `NA`. 

In [303]:
df.loc[df[df['Misc Feature'].isnull()].index, 'Misc Feature'] = 'None'

In [304]:
#total null values
print(df['Misc Feature'].isnull().sum())

#ensure that the NA values are in the value counts
df['Misc Feature'].value_counts()

0


None    1875
Shed      54
Gar2       4
Othr       2
Elev       1
TenC       1
Name: Misc Feature, dtype: int64

__Handling of `Pool QC` column__

In [305]:
#total non-null values
df['Pool QC'].value_counts().sum()

9

In [306]:
#total null values
df['Pool QC'].isnull().sum()

1928

In [307]:
df['Pool QC'].value_counts()

Gd    4
Fa    2
TA    2
Ex    1
Name: Pool QC, dtype: int64

In [308]:
df[df['Pool QC'].isna() & (df["Pool Area"] == 0)].shape

(1928, 80)

Similar to `Fence` column, `Pool QC` could be one of the good predictors of price. Furthermore, since `Pool Area = 0` also equal to each other, we are keeping it by assigning another neutral category for ordinal encoding later on. 

In [309]:
df.loc[df[df['Pool QC'].isnull()].index, 'Pool QC'] = 'None'

In [310]:
#total null values
print(df['Pool QC'].isnull().sum())

#ensure that the NA values are in the value counts
df['Pool QC'].value_counts()

0


None    1928
Gd         4
Fa         2
TA         2
Ex         1
Name: Pool QC, dtype: int64

In [311]:
#get null values of all columns again
df_null = pd.DataFrame(df.isnull().sum(), columns=['null_count']).sort_values(by='null_count',ascending=False)
df_null = df_null[df_null['null_count'] > 0].sort_values('null_count', ascending=False).sort_index()
#confirm no more null values 
df_null

,null_count


In [312]:
#double check the all df columns to see if number of entries all tally
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1937 entries, 0 to 2050
Data columns (total 80 columns):
Id                 1937 non-null int64
PID                1937 non-null int64
MS SubClass        1937 non-null int64
MS Zoning          1937 non-null object
Lot Frontage       1937 non-null float64
Lot Area           1937 non-null int64
Street             1937 non-null object
Lot Shape          1937 non-null object
Land Contour       1937 non-null object
Utilities          1937 non-null object
Lot Config         1937 non-null object
Land Slope         1937 non-null object
Neighborhood       1937 non-null object
Condition 1        1937 non-null object
Condition 2        1937 non-null object
Bldg Type          1937 non-null object
House Style        1937 non-null object
Overall Qual       1937 non-null int64
Overall Cond       1937 non-null int64
Year Built         1937 non-null int64
Year Remod/Add     1937 non-null int64
Roof Style         1937 non-null object
Roof Matl          1

Creating a function to get dataframe difference which might be used continually during data cleaning:

In [115]:
def get_diff_df(df1, df2, show_df, col_names=None):
    df1_index = set(df1.index)
    df2_index = set(df2.index)
    #set_diff
    set_diff = df1_index ^ df2_index
    if col_names == None:
        return show_df.loc[set_diff, :]
    return show_df.loc[set_diff, col_names]

__Changing of `MS Zoning` categories to be shorter for brevity__

In [313]:
df.groupby('MS Zoning').count()['Id']

MS Zoning
A (agr)       2
C (all)      11
FV          101
I (all)       1
RH           11
RL         1531
RM          280
Name: Id, dtype: int64

In [314]:
df.loc[df[df['MS Zoning'] == 'A (agr)'].index, 'MS Zoning'] = 'A'

In [315]:
df.loc[df[df['MS Zoning'] == 'C (all)'].index, 'MS Zoning'] = 'C'
df.loc[df[df['MS Zoning'] == 'I (all)'].index, 'MS Zoning'] = 'I'

In [316]:
df.groupby('MS Zoning').count()['Id']

MS Zoning
A        2
C       11
FV     101
I        1
RH      11
RL    1531
RM     280
Name: Id, dtype: int64

### Saving cleaned training data into a csv

In [317]:
df.to_csv('./datasets/clean_train.csv', index=False)

## Data Cleaning - test set

In [318]:
#read in dataset
test = pd.read_csv('./datasets/test.csv')

In [319]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 80 columns):
Id                 879 non-null int64
PID                879 non-null int64
MS SubClass        879 non-null int64
MS Zoning          879 non-null object
Lot Frontage       719 non-null float64
Lot Area           879 non-null int64
Street             879 non-null object
Alley              58 non-null object
Lot Shape          879 non-null object
Land Contour       879 non-null object
Utilities          879 non-null object
Lot Config         879 non-null object
Land Slope         879 non-null object
Neighborhood       879 non-null object
Condition 1        879 non-null object
Condition 2        879 non-null object
Bldg Type          879 non-null object
House Style        879 non-null object
Overall Qual       879 non-null int64
Overall Cond       879 non-null int64
Year Built         879 non-null int64
Year Remod/Add     879 non-null int64
Roof Style         879 non-null object
Roof M

In [320]:
test.shape

(879, 80)

### Null values - Numerical columns

We shall deal with the __numerical__ Null values first:

In [321]:
num_col = ['Id', 'PID', 'MS SubClass', 'Lot Frontage', 'Lot Area', 'Overall Qual', 'Overall Cond', 'Year Built',
           'Year Remod/Add', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 
           '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 
           'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces', 'Garage Yr Blt',
           'Garage Cars', 'Garage Area', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch',
           'Pool Area', 'Misc Val', 'Mo Sold', 'Yr Sold']

In [322]:
#get null count
test_null_num = pd.DataFrame(test[num_col].isnull().sum(), columns=['null_count'])

In [323]:
test_null_num = test_null_num[test_null_num['null_count'] > 0].sort_values('null_count', ascending=False).sort_index()
test_null_num

,null_count
Garage Yr Blt,45
Lot Frontage,160
Mas Vnr Area,1


__Handling of `Garage Yr Blt` column__

In [324]:
test[test['Garage Yr Blt'].isna()].shape

(45, 80)

Since this is the test set and we cannot drop any rows, we have to handle this column differently than the training set. We shall assign these values to 0 as they are only a fraction of the observations. 

In [325]:
test.loc[test[test['Garage Yr Blt'].isna()].index, 'Garage Yr Blt'] = 0

In [326]:
#confirm that the NA has been changed
test[test['Garage Yr Blt'].isna()].shape

(0, 80)

__Handling of `Lot Frontage` column__

In [327]:
test[test['Lot Frontage'].isna()].shape

(160, 80)

Since this is a continuous variable we can simply just assign those NA values to 0. 

In [328]:
test.loc[test[test['Lot Frontage'].isna()].index, 'Lot Frontage'] = 0

In [329]:
#confirm that the NA has been changed
test[test['Lot Frontage'].isna()].shape

(0, 80)

__Handling of `Mas Vnr Area` column__

In [330]:
test[test['Mas Vnr Area'].isna()].shape

(1, 80)

Since this is a continuous variable we can simply just assign those NA values to 0. 

In [331]:
test.loc[test[test['Mas Vnr Area'].isna()].index, 'Mas Vnr Area'] = 0

In [332]:
#confirm that the NA has been changed
test[test['Mas Vnr Area'].isna()].shape

(0, 80)

In [333]:
#get null count and confirm if all NA has been accounted for in numerical columns
test_null_num = pd.DataFrame(test[num_col].isnull().sum(), columns=['null_count'])
test_null_num = test_null_num[test_null_num['null_count'] > 0].sort_values('null_count', ascending=False).sort_index()
test_null_num

,null_count


### Null values - Categorical columns

In [334]:
#get null count for categorical columns
test_null_cat = pd.DataFrame(test.isnull().sum(), columns=['null_count'])
test_null_cat = test_null_cat[test_null_cat['null_count'] > 0].sort_values('null_count', ascending=False).sort_index()
test_null_cat

,null_count
Alley,821
Bsmt Cond,25
Bsmt Exposure,25
Bsmt Qual,25
BsmtFin Type 1,25
BsmtFin Type 2,25
Electrical,1
Fence,707
Fireplace Qu,422
Garage Cond,45


__Handling of `Alley` column__

In [335]:
#total non-null values for Alley:
test['Alley'].value_counts()

Grvl    35
Pave    23
Name: Alley, dtype: int64

In [336]:
#total null values for Alley:
test['Alley'].isnull().sum()

821

In [337]:
#Percentage of null values in Alley
test['Alley'].isnull().sum()/test.shape[0]

0.9340159271899886

Since there are too many null values (almost 93% of the total observations) in `Alley`, and unlikely will influence the house price, we shall drop it.   

In [338]:
test.drop(columns='Alley', inplace=True)

In [339]:
test.shape

(879, 79)

__Handling of `Bsmt Cond` column__

In [340]:
#total non-null values
print('non-null values', test['Bsmt Cond'].value_counts().sum())
#total null values
print('null values', test['Bsmt Cond'].isnull().sum())
#breakdown value counts
test['Bsmt Cond'].value_counts()

non-null values 854
null values 25


TA    782
Fa     39
Gd     33
Name: Bsmt Cond, dtype: int64

Since `Bsmt Cond` column has very few null values, we will be keeping it by assigning another neutral category for ordinal encoding later on.

In [341]:
#assign NA to null values
test.loc[test[test['Bsmt Cond'].isnull()].index, 'Bsmt Cond'] = 'None'
#total null values
print(test['Bsmt Cond'].isnull().sum())
#ensure that the NA values are in the value counts
test['Bsmt Cond'].value_counts()

0


TA      782
Fa       39
Gd       33
None     25
Name: Bsmt Cond, dtype: int64

__Handling of `Bsmt Exposure` column__

In [342]:
#total non-null values
print('non-null values', test['Bsmt Exposure'].value_counts().sum())
#total null values
print('null values', test['Bsmt Exposure'].isnull().sum())
#breakdown value counts
test['Bsmt Exposure'].value_counts()

non-null values 854
null values 25


No    567
Av    130
Gd     81
Mn     76
Name: Bsmt Exposure, dtype: int64

Since `Bsmt Exposure` column has very few null values, we will be keeping it by assigning another neutral category for ordinal encoding later on.

In [343]:
#assign NA to null values
test.loc[test[test['Bsmt Exposure'].isnull()].index, 'Bsmt Exposure'] = 'None'
#total null values
print(test['Bsmt Exposure'].isnull().sum())
#ensure that the NA values are in the value counts
test['Bsmt Exposure'].value_counts()

0


No      567
Av      130
Gd       81
Mn       76
None     25
Name: Bsmt Exposure, dtype: int64

__Handling of `Bsmt Qual` column__

In [344]:
#total non-null values
print('non-null values', test['Bsmt Qual'].value_counts().sum())
#total null values
print('null values', test['Bsmt Qual'].isnull().sum())
#breakdown value counts
test['Bsmt Qual'].value_counts()

non-null values 854
null values 25


TA    396
Gd    355
Ex     74
Fa     28
Po      1
Name: Bsmt Qual, dtype: int64

Since `Bsmt Qual` column has very few null values, we will be keeping it by assigning another neutral category for ordinal encoding later on.

In [345]:
#assign NA to null values
test.loc[test[test['Bsmt Qual'].isnull()].index, 'Bsmt Qual'] = 'None'
#total null values
print(test['Bsmt Qual'].isnull().sum())
#ensure that the NA values are in the value counts
test['Bsmt Qual'].value_counts()

0


TA      396
Gd      355
Ex       74
Fa       28
None     25
Po        1
Name: Bsmt Qual, dtype: int64

__Handling of `BsmtFin Type 1` column__

In [346]:
#total non-null values
print('non-null values', test['BsmtFin Type 1'].value_counts().sum())
#total null values
print('null values', test['BsmtFin Type 1'].isnull().sum())
#breakdown value counts
test['BsmtFin Type 1'].value_counts()

non-null values 854
null values 25


Unf    248
GLQ    244
ALQ    136
Rec    105
BLQ     69
LwQ     52
Name: BsmtFin Type 1, dtype: int64

Since `BsmtFin Type 1` column has very few null values, we will be keeping it by assigning another neutral category for ordinal encoding later on.

In [347]:
#assign NA to null values
test.loc[test[test['BsmtFin Type 1'].isnull()].index, 'BsmtFin Type 1'] = 'None'
#total null values
print(test['BsmtFin Type 1'].isnull().sum())
#ensure that the NA values are in the value counts
test['BsmtFin Type 1'].value_counts()

0


Unf     248
GLQ     244
ALQ     136
Rec     105
BLQ      69
LwQ      52
None     25
Name: BsmtFin Type 1, dtype: int64

__Handling of `BsmtFin Type 2` column__

In [348]:
#total non-null values
print('non-null values', test['BsmtFin Type 2'].value_counts().sum())
#total null values
print('null values', test['BsmtFin Type 2'].isnull().sum())
#breakdown value counts
test['BsmtFin Type 2'].value_counts()

non-null values 854
null values 25


Unf    750
LwQ     29
Rec     26
BLQ     20
ALQ     18
GLQ     11
Name: BsmtFin Type 2, dtype: int64

Since `BsmtFin Type 2` column has very few null values, we will be keeping it by assigning another neutral category for ordinal encoding later on.

In [349]:
#assign NA to null values
test.loc[test[test['BsmtFin Type 2'].isnull()].index, 'BsmtFin Type 2'] = 'None'
#total null values
print(test['BsmtFin Type 2'].isnull().sum())
#ensure that the NA values are in the value counts
test['BsmtFin Type 2'].value_counts()

0


Unf     750
LwQ      29
Rec      26
None     25
BLQ      20
ALQ      18
GLQ      11
Name: BsmtFin Type 2, dtype: int64

__Handling of `Electrical` column__

In [350]:
#total non-null values
print('non-null values', test['Electrical'].value_counts().sum())
#total null values
print('null values', test['Electrical'].isnull().sum())
#breakdown value counts
test['Electrical'].value_counts()

non-null values 878
null values 1


SBrkr    814
FuseA     48
FuseF     15
FuseP      1
Name: Electrical, dtype: int64

In [351]:
#null value neighborhood
test[test['Electrical'].isnull()]['Neighborhood']

635    Timber
Name: Neighborhood, dtype: object

In [352]:
#find the most frequently occuring Electrical by neighborhoods
test.groupby('Neighborhood')['Electrical'].describe()

,count,unique,top,freq
Neighborhood,,,,
Blmngtn,6,1,SBrkr,6
Blueste,4,1,SBrkr,4
BrDale,11,1,SBrkr,11
BrkSide,32,3,SBrkr,24
ClearCr,17,1,SBrkr,17
CollgCr,87,1,SBrkr,87
Crawfor,32,3,SBrkr,28
Edwards,51,3,SBrkr,42
Gilbert,49,2,SBrkr,48


Since `Electrical` column has very one null value, we will be keeping it. But there is no None column so we will use the most frequently occuring `Electrical` for the neighborhood, which is 'SBrkr'. 

In [353]:
#assign NA to null values
test.loc[test[test['Electrical'].isnull()].index, 'Electrical'] = 'SBrkr'
#total null values
print(test['Electrical'].isnull().sum())
#ensure that the NA values are in the value counts
test['Electrical'].value_counts()

0


SBrkr    815
FuseA     48
FuseF     15
FuseP      1
Name: Electrical, dtype: int64

__Handling of `Fence` column__

In [354]:
#total non-null values
print('non-null values', test['Fence'].value_counts().sum())
#total null values
print('null values', test['Fence'].isnull().sum())
#breakdown value counts
test['Fence'].value_counts()

non-null values 172
null values 707


MnPrv    103
GdPrv     35
GdWo      32
MnWw       2
Name: Fence, dtype: int64

Even though `Fence` column has many NA values, it could be one of the good predictors of price, so we are keeping it by assigning another neutral category for ordinal encoding later on. 

In [355]:
#assign NA to null values
test.loc[test[test['Fence'].isnull()].index, 'Fence'] = 'None'
#total null values
print(test['Fence'].isnull().sum())
#ensure that the NA values are in the value counts
test['Fence'].value_counts()

0


None     707
MnPrv    103
GdPrv     35
GdWo      32
MnWw       2
Name: Fence, dtype: int64

__Handling of `Fireplace Qu` column__

In [356]:
#total non-null values
print('non-null values', test['Fireplace Qu'].value_counts().sum())
#total null values
print('null values', test['Fireplace Qu'].isnull().sum())
#breakdown value counts
test['Fireplace Qu'].value_counts()

non-null values 457
null values 422


Gd    221
TA    193
Fa     16
Po     15
Ex     12
Name: Fireplace Qu, dtype: int64

Even though `Fireplace Qu` column has many NA values, it could be one of the good predictors of price, so we are keeping it by assigning another neutral category for ordinal encoding later on. 

In [357]:
#assign NA to null values
test.loc[test[test['Fireplace Qu'].isnull()].index, 'Fireplace Qu'] = 'None'
#total null values
print(test['Fireplace Qu'].isnull().sum())
#ensure that the NA values are in the value counts
test['Fireplace Qu'].value_counts()

0


None    422
Gd      221
TA      193
Fa       16
Po       15
Ex       12
Name: Fireplace Qu, dtype: int64

__Handling of `Garage Cond` column__

In [358]:
#total non-null values
print('non-null values', test['Garage Cond'].value_counts().sum())
#total null values
print('null values', test['Garage Cond'].isnull().sum())
#breakdown value counts
test['Garage Cond'].value_counts()

non-null values 834
null values 45


TA    797
Fa     27
Po      6
Gd      3
Ex      1
Name: Garage Cond, dtype: int64

Even though `Garage Cond` column has few NA values, and it could be one of the good predictors of price, so we are keeping it by assigning another neutral category for ordinal encoding later on. 

In [359]:
#assign NA to null values
test.loc[test[test['Garage Cond'].isnull()].index, 'Garage Cond'] = 'None'
#total null values
print(test['Garage Cond'].isnull().sum())
#ensure that the NA values are in the value counts
test['Garage Cond'].value_counts()

0


TA      797
None     45
Fa       27
Po        6
Gd        3
Ex        1
Name: Garage Cond, dtype: int64

__Handling of `Garage Finish` column__

In [360]:
#total non-null values
print('non-null values', test['Garage Finish'].value_counts().sum())
#total null values
print('null values', test['Garage Finish'].isnull().sum())
#breakdown value counts
test['Garage Finish'].value_counts()

non-null values 834
null values 45


Unf    382
RFn    233
Fin    219
Name: Garage Finish, dtype: int64

Even though `Garage Finish` column has few NA values, and it could be one of the good predictors of price, so we are keeping it by assigning another neutral category for ordinal encoding later on. 

In [361]:
#assign NA to null values
test.loc[test[test['Garage Finish'].isnull()].index, 'Garage Finish'] = 'None'
#total null values
print(test['Garage Finish'].isnull().sum())
#ensure that the NA values are in the value counts
test['Garage Finish'].value_counts()

0


Unf     382
RFn     233
Fin     219
None     45
Name: Garage Finish, dtype: int64

__Handling of `Garage Qual` column__

In [362]:
#total non-null values
print('non-null values', test['Garage Qual'].value_counts().sum())
#total null values
print('null values', test['Garage Qual'].isnull().sum())
#breakdown value counts
test['Garage Qual'].value_counts()

non-null values 834
null values 45


TA    783
Fa     42
Gd      6
Po      3
Name: Garage Qual, dtype: int64

Even though `Garage Qual` column has few NA values, and it could be one of the good predictors of price, so we are keeping it by assigning another neutral category for ordinal encoding later on. 

In [363]:
#assign NA to null values
test.loc[test[test['Garage Qual'].isnull()].index, 'Garage Qual'] = 'None'
#total null values
print(test['Garage Qual'].isnull().sum())
#ensure that the NA values are in the value counts
test['Garage Qual'].value_counts()

0


TA      783
None     45
Fa       42
Gd        6
Po        3
Name: Garage Qual, dtype: int64

__Handling of `Garage Type` column__

In [364]:
#total non-null values
print('non-null values', test['Garage Type'].value_counts().sum())
#total null values
print('null values', test['Garage Type'].isnull().sum())
#breakdown value counts
test['Garage Type'].value_counts()

non-null values 835
null values 44


Attchd     518
Detchd     246
BuiltIn     54
Basment      9
2Types       4
CarPort      4
Name: Garage Type, dtype: int64

Even though `Garage Type` column has few NA values, and it could be one of the good predictors of price, so we are keeping it by assigning another neutral category for ordinal encoding later on. 

In [365]:
#assign NA to null values
test.loc[test[test['Garage Type'].isnull()].index, 'Garage Type'] = 'None'
#total null values
print(test['Garage Type'].isnull().sum())
#ensure that the NA values are in the value counts
test['Garage Type'].value_counts()

0


Attchd     518
Detchd     246
BuiltIn     54
None        44
Basment      9
2Types       4
CarPort      4
Name: Garage Type, dtype: int64

__Handling of `Mas Vnr Type` column__

In [366]:
#total non-null values
print('non-null values', test['Mas Vnr Type'].value_counts().sum())
#total null values
print('null values', test['Mas Vnr Type'].isnull().sum())
#breakdown value counts
test['Mas Vnr Type'].value_counts()

non-null values 878
null values 1


None       534
BrkFace    250
Stone       81
BrkCmn      12
CBlock       1
Name: Mas Vnr Type, dtype: int64

Even though `Mas Vnr Type` has only column has few NA values, and it could be one of the good predictors of price, so we are keeping it by assigning another neutral category for encoding later on. 

In [367]:
#assign NA to null values
test.loc[test[test['Mas Vnr Type'].isnull()].index, 'Mas Vnr Type'] = 'None'
#total null values

print(test['Mas Vnr Type'].isnull().sum())
#ensure that the NA values are in the value counts
test['Mas Vnr Type'].value_counts()

0


None       535
BrkFace    250
Stone       81
BrkCmn      12
CBlock       1
Name: Mas Vnr Type, dtype: int64

__Handling of `Misc Feature` column__

In [368]:
#total non-null values
print('non-null values', test['Misc Feature'].value_counts().sum())
#total null values
print('null values', test['Misc Feature'].isnull().sum())
#breakdown value counts
test['Misc Feature'].value_counts()

non-null values 41
null values 838


Shed    39
Gar2     1
Othr     1
Name: Misc Feature, dtype: int64

Even though `Misc Feature` has many rows that this column has face to face. NA values, and it could be one of the good predictors of price, so we are keeping it by assigning another neutral category for encoding later on. 

In [369]:
#assign NA to null values
test.loc[test[test['Misc Feature'].isnull()].index, 'Misc Feature'] = 'None'
#total null values

print(test['Misc Feature'].isnull().sum())
#ensure that the NA values are in the value counts
test['Misc Feature'].value_counts()

0


None    838
Shed     39
Gar2      1
Othr      1
Name: Misc Feature, dtype: int64

__Handling of `Pool QC` column__

In [370]:
#total non-null values
print('non-null values', test['Pool QC'].value_counts().sum())
#total null values
print('null values', test['Pool QC'].isnull().sum())
#breakdown value counts
test['Pool QC'].value_counts()

non-null values 4
null values 875


Ex    3
TA    1
Name: Pool QC, dtype: int64

Even though `Pool QC` has only column has few NA values, and it could be one of the good predictors of price, so we are keeping it by assigning another neutral category for ordinal encoding later on. 

In [371]:
#assign NA to null values
test.loc[test[test['Pool QC'].isnull()].index, 'Pool QC'] = 'None'
#total null values

print(test['Pool QC'].isnull().sum())
#ensure that the NA values are in the value counts
test['Pool QC'].value_counts()

0


None    875
Ex        3
TA        1
Name: Pool QC, dtype: int64

In [372]:
#ensure no more null values
#get null count for categorical columns
test_null_cat = pd.DataFrame(test.isnull().sum(), columns=['null_count'])
test_null_cat = test_null_cat[test_null_cat['null_count'] > 0].sort_values('null_count', ascending=False).sort_index()
test_null_cat

,null_count


In [373]:
test.shape

(879, 79)

In [374]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 79 columns):
Id                 879 non-null int64
PID                879 non-null int64
MS SubClass        879 non-null int64
MS Zoning          879 non-null object
Lot Frontage       879 non-null float64
Lot Area           879 non-null int64
Street             879 non-null object
Lot Shape          879 non-null object
Land Contour       879 non-null object
Utilities          879 non-null object
Lot Config         879 non-null object
Land Slope         879 non-null object
Neighborhood       879 non-null object
Condition 1        879 non-null object
Condition 2        879 non-null object
Bldg Type          879 non-null object
House Style        879 non-null object
Overall Qual       879 non-null int64
Overall Cond       879 non-null int64
Year Built         879 non-null int64
Year Remod/Add     879 non-null int64
Roof Style         879 non-null object
Roof Matl          879 non-null object
Exter

### Saving cleaned test data into a csv

In [375]:
test.to_csv('./datasets/clean_test.csv', index=False)

## Combining both test and train

In [395]:
train_df = pd.read_csv('./datasets/clean_train.csv')
test_df = pd.read_csv('./datasets/clean_test.csv')

#train set has one more column which is the SalePrice 
print(train_df.shape)
print(test_df.shape)

(1937, 80)
(879, 79)


In [396]:
#store train set sale price seperately
train_sale_price = train_df[['SalePrice']]
train_sale_price.head()

,SalePrice
0,130500
1,220000
2,109000
3,174000
4,138500


In [397]:
#drop train set sale price
train_df.drop(columns='SalePrice', inplace=True)

In [398]:
#now both train set and test set have the same number of columns
print(train_df.shape)
test_df.shape

(1937, 79)


(879, 79)

In [399]:
comb_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [400]:
comb_df.shape

(2816, 79)

In [401]:
#ensure no more null values
#get null count for ALL columns
comb_null = pd.DataFrame(comb_df.isnull().sum(), columns=['null_count'])
comb_null = comb_null[comb_null['null_count'] > 0].sort_values('null_count', ascending=False).sort_index()
comb_null

,null_count


__Changing of `MS Zoning` categories to be shorter for brevity__

In [424]:
comb_df.groupby('MS Zoning').count()['Id']

MS Zoning
A             2
C            11
C (all)       6
FV          139
I             1
I (all)       1
RH           24
RL         2206
RM          426
Name: Id, dtype: int64

In [426]:
comb_df.loc[comb_df[comb_df['MS Zoning'] == 'A (agr)'].index, 'MS Zoning'] = 'A'

In [427]:
comb_df.loc[comb_df[comb_df['MS Zoning'] == 'C (all)'].index, 'MS Zoning'] = 'C'
comb_df.loc[comb_df[comb_df['MS Zoning'] == 'I (all)'].index, 'MS Zoning'] = 'I'

In [428]:
comb_df.groupby('MS Zoning').count()['Id']

MS Zoning
A        2
C       17
FV     139
I        2
RH      24
RL    2206
RM     426
Name: Id, dtype: int64

In [429]:
#save it to csv
comb_df.to_csv('./datasets/clean_comb.csv', index=False)

## Encoding of categorical data

In [479]:
#read the comb csv
comb = pd.read_csv('./datasets/clean_comb.csv')

In [480]:
#confirm again no null values
comb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2816 entries, 0 to 2815
Data columns (total 79 columns):
Id                 2816 non-null int64
PID                2816 non-null int64
MS SubClass        2816 non-null int64
MS Zoning          2816 non-null object
Lot Frontage       2816 non-null float64
Lot Area           2816 non-null int64
Street             2816 non-null object
Lot Shape          2816 non-null object
Land Contour       2816 non-null object
Utilities          2816 non-null object
Lot Config         2816 non-null object
Land Slope         2816 non-null object
Neighborhood       2816 non-null object
Condition 1        2816 non-null object
Condition 2        2816 non-null object
Bldg Type          2816 non-null object
House Style        2816 non-null object
Overall Qual       2816 non-null int64
Overall Cond       2816 non-null int64
Year Built         2816 non-null int64
Year Remod/Add     2816 non-null int64
Roof Style         2816 non-null object
Roof Matl          2

In [481]:
cat_var = [i for i in comb.columns if comb[i].dtype == object ]
cat_var

['MS Zoning',
 'Street',
 'Lot Shape',
 'Land Contour',
 'Utilities',
 'Lot Config',
 'Land Slope',
 'Neighborhood',
 'Condition 1',
 'Condition 2',
 'Bldg Type',
 'House Style',
 'Roof Style',
 'Roof Matl',
 'Exterior 1st',
 'Exterior 2nd',
 'Mas Vnr Type',
 'Exter Qual',
 'Exter Cond',
 'Foundation',
 'Bsmt Qual',
 'Bsmt Cond',
 'Bsmt Exposure',
 'BsmtFin Type 1',
 'BsmtFin Type 2',
 'Heating',
 'Heating QC',
 'Central Air',
 'Electrical',
 'Kitchen Qual',
 'Functional',
 'Fireplace Qu',
 'Garage Type',
 'Garage Finish',
 'Garage Qual',
 'Garage Cond',
 'Paved Drive',
 'Pool QC',
 'Fence',
 'Misc Feature',
 'Sale Type']

In [482]:
#ensure no more null values
#get null count for ALL columns
comb_null = pd.DataFrame(comb.isnull().sum(), columns=['null_count'])
comb_null = comb_null[comb_null['null_count'] > 0].sort_values('null_count', ascending=False).sort_index()
comb_null

,null_count


We will do one-hot encoding for nominal variables and ordinal encoding for ordinal variables. 

__One-hot encoding for `MS Zoning` column__

In [483]:
comb = pd.get_dummies(comb, columns=['MS Zoning'], prefix='zoning', drop_first=True)

In [484]:
print(comb.shape)
comb.head()

(2816, 84)


,Id,PID,MS SubClass,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,zoning_C,zoning_FV,zoning_I,zoning_RH,zoning_RL,zoning_RM
0,109,533352170,60,0.0,13517,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,None,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,None,None,None,0,3,2010,WD,0,0,0,0,1,0
1,544,531379050,60,43.0,11492,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,None,None,None,0,4,2009,WD,0,0,0,0,1,0
2,153,535304180,20,68.0,7922,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,None,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,None,None,None,0,1,2010,WD,0,0,0,0,1,0
3,318,916386060,60,73.0,9802,Pave,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,SBrkr,744,700,0,1444,0.0,0.0,2,1,3,1,TA,7,Typ,0,None,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,None,None,None,0,4,2010,WD,0,0,0,0,1,0
4,255,906425045,50,82.0,14235,Pave,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA,TA,Y,SBrkr,831,614,0,1445,0.0,0.0,2,0,3,1,TA,6,Typ,0,None,Detchd,1957.0,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,None,None,None,0,3,2010,WD,0,0,0,0,1,0


__One-hot encoding for `Street` column__

In [485]:
comb.Street.value_counts()

Pave    2806
Grvl      10
Name: Street, dtype: int64

In [486]:
#One-hot encoding for Street column
comb = pd.get_dummies(comb, columns=['Street'], prefix='street', drop_first=True)

In [487]:
comb.shape

(2816, 84)

In [488]:
comb.head()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,zoning_C,zoning_FV,zoning_I,zoning_RH,zoning_RL,zoning_RM,street_Pave
0,109,533352170,60,0.0,13517,IR1,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,None,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,None,None,None,0,3,2010,WD,0,0,0,0,1,0,1
1,544,531379050,60,43.0,11492,IR1,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,None,None,None,0,4,2009,WD,0,0,0,0,1,0,1
2,153,535304180,20,68.0,7922,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,None,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,None,None,None,0,1,2010,WD,0,0,0,0,1,0,1
3,318,916386060,60,73.0,9802,Reg,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,SBrkr,744,700,0,1444,0.0,0.0,2,1,3,1,TA,7,Typ,0,None,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,None,None,None,0,4,2010,WD,0,0,0,0,1,0,1
4,255,906425045,50,82.0,14235,IR1,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA,TA,Y,SBrkr,831,614,0,1445,0.0,0.0,2,0,3,1,TA,6,Typ,0,None,Detchd,1957.0,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,None,None,None,0,3,2010,WD,0,0,0,0,1,0,1


__Ordinal encoding for `Lot Shape` column__

In [489]:
comb.groupby('Lot Shape').count()['Id']

Lot Shape
IR1     963
IR2      76
IR3      16
Reg    1761
Name: Id, dtype: int64

In [490]:
def lot_shape(x):
    if x == 'IR3':
        return 0
    elif x == 'IR2':
        return 1 
    elif x == 'IR1':
        return 2 
    elif x == 'Reg':
        return 3

comb['Lot Shape'] = comb['Lot Shape'].map(lot_shape)

In [491]:
print(comb.shape)
comb.head()

(2816, 84)


,Id,PID,MS SubClass,Lot Frontage,Lot Area,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,zoning_C,zoning_FV,zoning_I,zoning_RH,zoning_RL,zoning_RM,street_Pave
0,109,533352170,60,0.0,13517,2,Lvl,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,None,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,None,None,None,0,3,2010,WD,0,0,0,0,1,0,1
1,544,531379050,60,43.0,11492,2,Lvl,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,None,None,None,0,4,2009,WD,0,0,0,0,1,0,1
2,153,535304180,20,68.0,7922,3,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,None,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,None,None,None,0,1,2010,WD,0,0,0,0,1,0,1
3,318,916386060,60,73.0,9802,3,Lvl,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,SBrkr,744,700,0,1444,0.0,0.0,2,1,3,1,TA,7,Typ,0,None,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,None,None,None,0,4,2010,WD,0,0,0,0,1,0,1
4,255,906425045,50,82.0,14235,2,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA,TA,Y,SBrkr,831,614,0,1445,0.0,0.0,2,0,3,1,TA,6,Typ,0,None,Detchd,1957.0,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,None,None,None,0,3,2010,WD,0,0,0,0,1,0,1


__One-hot encoding for `Land Contour` column__

In [492]:
comb['Land Contour'].value_counts()

Lvl    2534
HLS     117
Bnk     109
Low      56
Name: Land Contour, dtype: int64

In [493]:
comb = pd.get_dummies(comb, columns=['Land Contour'], prefix='land_countour', drop_first=True)

In [494]:
print(comb.shape)
comb.head()

(2816, 86)


,Id,PID,MS SubClass,Lot Frontage,Lot Area,Lot Shape,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,zoning_C,zoning_FV,zoning_I,zoning_RH,zoning_RL,zoning_RM,street_Pave,land_countour_HLS,land_countour_Low,land_countour_Lvl
0,109,533352170,60,0.0,13517,2,AllPub,CulDSac,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,None,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,None,None,None,0,3,2010,WD,0,0,0,0,1,0,1,0,0,1
1,544,531379050,60,43.0,11492,2,AllPub,CulDSac,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,None,None,None,0,4,2009,WD,0,0,0,0,1,0,1,0,0,1
2,153,535304180,20,68.0,7922,3,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,None,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,None,None,None,0,1,2010,WD,0,0,0,0,1,0,1,0,0,1
3,318,916386060,60,73.0,9802,3,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,SBrkr,744,700,0,1444,0.0,0.0,2,1,3,1,TA,7,Typ,0,None,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,None,None,None,0,4,2010,WD,0,0,0,0,1,0,1,0,0,1
4,255,906425045,50,82.0,14235,2,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA,TA,Y,SBrkr,831,614,0,1445,0.0,0.0,2,0,3,1,TA,6,Typ,0,None,Detchd,1957.0,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,None,None,None,0,3,2010,WD,0,0,0,0,1,0,1,0,0,1


__Ordinal encoding for `Utilities` column__

In [495]:
comb.groupby('Utilities').count()['Id']

Utilities
AllPub    2813
NoSeWa       1
NoSewr       2
Name: Id, dtype: int64

In [496]:
def util(x):
    if x == 'ELO':
        return 0
    elif x == 'NoSeWa':
        return 1 
    elif x == 'NoSewr':
        return 2 
    elif x == 'AllPub':
        return 3

comb['Utilities'] = comb['Utilities'].map(util)

In [497]:
comb['Utilities'].dtype

dtype('int64')

In [498]:
comb.groupby('Utilities').count()['Id']

Utilities
1       1
2       2
3    2813
Name: Id, dtype: int64

In [499]:
comb.shape

(2816, 86)

__One-hot encoding for `Lot Config` column__

In [501]:
comb['Lot Config'].value_counts()

Inside     2049
Corner      494
CulDSac     178
FR2          81
FR3          14
Name: Lot Config, dtype: int64

In [502]:
comb = pd.get_dummies(comb, columns=['Lot Config'], prefix='lot_conf', drop_first=True)
comb.head()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Lot Shape,Utilities,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,zoning_C,zoning_FV,zoning_I,zoning_RH,zoning_RL,zoning_RM,street_Pave,land_countour_HLS,land_countour_Low,land_countour_Lvl,lot_conf_CulDSac,lot_conf_FR2,lot_conf_FR3,lot_conf_Inside
0,109,533352170,60,0.0,13517,2,3,Gtl,Sawyer,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,None,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,None,None,None,0,3,2010,WD,0,0,0,0,1,0,1,0,0,1,1,0,0,0
1,544,531379050,60,43.0,11492,2,3,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,None,None,None,0,4,2009,WD,0,0,0,0,1,0,1,0,0,1,1,0,0,0
2,153,535304180,20,68.0,7922,3,3,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,None,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,None,None,None,0,1,2010,WD,0,0,0,0,1,0,1,0,0,1,0,0,0,1
3,318,916386060,60,73.0,9802,3,3,Gtl,Timber,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,SBrkr,744,700,0,1444,0.0,0.0,2,1,3,1,TA,7,Typ,0,None,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,None,None,None,0,4,2010,WD,0,0,0,0,1,0,1,0,0,1,0,0,0,1
4,255,906425045,50,82.0,14235,2,3,Gtl,SawyerW,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA,TA,Y,SBrkr,831,614,0,1445,0.0,0.0,2,0,3,1,TA,6,Typ,0,None,Detchd,1957.0,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,None,None,None,0,3,2010,WD,0,0,0,0,1,0,1,0,0,1,0,0,0,1


__Ordinal encoding for `Land Slope` column__

In [503]:
comb['Land Slope'].value_counts()

Gtl    2679
Mod     122
Sev      15
Name: Land Slope, dtype: int64

In [504]:
def conv_ordinal(x):
    if x == 'Sev':
        return 0
    elif x == 'Mod':
        return 1 
    elif x == 'Gtl':
        return 2 

comb['Land Slope'] = comb['Land Slope'].map(conv_ordinal)

In [505]:
comb['Land Slope'].value_counts()

2    2679
1     122
0      15
Name: Land Slope, dtype: int64

__One-hot encoding for `Neighborhood` column__

In [506]:
comb['Neighborhood'].value_counts()

NAmes      435
CollgCr    265
OldTown    220
Somerst    182
NridgHt    166
Gilbert    165
Edwards    164
Sawyer     147
NWAmes     131
SawyerW    119
Mitchel    111
Crawfor    103
BrkSide     99
IDOTRR      74
Timber      72
NoRidge     71
StoneBr     51
ClearCr     43
SWISU       43
MeadowV     30
BrDale      29
Blmngtn     28
Veenker     24
NPkVill     23
Blueste     10
Greens       8
GrnHill      2
Landmrk      1
Name: Neighborhood, dtype: int64

In [508]:
comb = pd.get_dummies(comb, columns=['Neighborhood'], prefix='neigh', drop_first=True)

In [511]:
print(comb.shape)
comb.head()

(2816, 115)


,Id,PID,MS SubClass,Lot Frontage,Lot Area,Lot Shape,Utilities,Land Slope,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,zoning_C,zoning_FV,zoning_I,zoning_RH,zoning_RL,zoning_RM,street_Pave,land_countour_HLS,land_countour_Low,land_countour_Lvl,lot_conf_CulDSac,lot_conf_FR2,lot_conf_FR3,lot_conf_Inside,neigh_Blueste,neigh_BrDale,neigh_BrkSide,neigh_ClearCr,neigh_CollgCr,neigh_Crawfor,neigh_Edwards,neigh_Gilbert,neigh_Greens,neigh_GrnHill,neigh_IDOTRR,neigh_Landmrk,neigh_MeadowV,neigh_Mitchel,neigh_NAmes,neigh_NPkVill,neigh_NWAmes,neigh_NoRidge,neigh_NridgHt,neigh_OldTown,neigh_SWISU,neigh_Sawyer,neigh_SawyerW,neigh_Somerst,neigh_StoneBr,neigh_Timber,neigh_Veenker
0,109,533352170,60,0.0,13517,2,3,2,RRAe,Norm,1Fam,2Story,6,8,1976,2005,Gable,CompShg,HdBoard,Plywood,BrkFace,289.0,Gd,TA,CBlock,TA,TA,No,GLQ,533.0,Unf,0.0,192.0,725.0,GasA,Ex,Y,SBrkr,725,754,0,1479,0.0,0.0,2,1,3,1,Gd,6,Typ,0,None,Attchd,1976.0,RFn,2.0,475.0,TA,TA,Y,0,44,0,0,0,0,None,None,None,0,3,2010,WD,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,544,531379050,60,43.0,11492,2,3,2,Norm,Norm,1Fam,2Story,7,5,1996,1997,Gable,CompShg,VinylSd,VinylSd,BrkFace,132.0,Gd,TA,PConc,Gd,TA,No,GLQ,637.0,Unf,0.0,276.0,913.0,GasA,Ex,Y,SBrkr,913,1209,0,2122,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1997.0,RFn,2.0,559.0,TA,TA,Y,0,74,0,0,0,0,None,None,None,0,4,2009,WD,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,153,535304180,20,68.0,7922,3,3,2,Norm,Norm,1Fam,1Story,5,7,1953,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,Gd,CBlock,TA,TA,No,GLQ,731.0,Unf,0.0,326.0,1057.0,GasA,TA,Y,SBrkr,1057,0,0,1057,1.0,0.0,1,0,3,1,Gd,5,Typ,0,None,Detchd,1953.0,Unf,1.0,246.0,TA,TA,Y,0,52,0,0,0,0,None,None,None,0,1,2010,WD,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,318,916386060,60,73.0,9802,3,3,2,Norm,Norm,1Fam,2Story,5,5,2006,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0.0,Unf,0.0,384.0,384.0,GasA,Gd,Y,SBrkr,744,700,0,1444,0.0,0.0,2,1,3,1,TA,7,Typ,0,None,BuiltIn,2007.0,Fin,2.0,400.0,TA,TA,Y,100,0,0,0,0,0,None,None,None,0,4,2010,WD,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,255,906425045,50,82.0,14235,2,3,2,Norm,Norm,1Fam,1.5Fin,6,8,1900,1993,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,PConc,Fa,Gd,No,Unf,0.0,Unf,0.0,676.0,676.0,GasA,TA,Y,SBrkr,831,614,0,1445,0.0,0.0,2,0,3,1,TA,6,Typ,0,None,Detchd,1957.0,Unf,2.0,484.0,TA,TA,N,0,59,0,0,0,0,None,None,None,0,3,2010,WD,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


__One-hot encoding for `Condition 1` column__

In [512]:
comb.groupby('Condition 1').count()['Id']

Condition 1
Artery      84
Feedr      151
Norm      2430
PosA        20
PosN        39
RRAe        27
RRAn        50
RRNe         6
RRNn         9
Name: Id, dtype: int64

In [514]:
comb = pd.get_dummies(comb, columns=['Condition 1'], prefix='cond1', drop_first=True)

In [515]:
comb.iloc[:,99:].head()

,neigh_MeadowV,neigh_Mitchel,neigh_NAmes,neigh_NPkVill,neigh_NWAmes,neigh_NoRidge,neigh_NridgHt,neigh_OldTown,neigh_SWISU,neigh_Sawyer,neigh_SawyerW,neigh_Somerst,neigh_StoneBr,neigh_Timber,neigh_Veenker,cond1_Feedr,cond1_Norm,cond1_PosA,cond1_PosN,cond1_RRAe,cond1_RRAn,cond1_RRNe,cond1_RRNn
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0


__One-hot encoding for `Condition 2` column__

In [516]:
comb.groupby('Condition 2').count()['Id']

Condition 2
Artery       4
Feedr       12
Norm      2788
PosA         4
PosN         4
RRAe         1
RRAn         1
RRNn         2
Name: Id, dtype: int64

In [517]:
comb = pd.get_dummies(comb, columns=['Condition 2'], prefix='cond2', drop_first=True)

In [518]:
comb.iloc[:,99:].head()

,neigh_Mitchel,neigh_NAmes,neigh_NPkVill,neigh_NWAmes,neigh_NoRidge,neigh_NridgHt,neigh_OldTown,neigh_SWISU,neigh_Sawyer,neigh_SawyerW,neigh_Somerst,neigh_StoneBr,neigh_Timber,neigh_Veenker,cond1_Feedr,cond1_Norm,cond1_PosA,cond1_PosN,cond1_RRAe,cond1_RRAn,cond1_RRNe,cond1_RRNn,cond2_Feedr,cond2_Norm,cond2_PosA,cond2_PosN,cond2_RRAe,cond2_RRAn,cond2_RRNn
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0


__One-hot encoding for other categorical columns__

In [519]:
comb = pd.get_dummies(comb, columns=['Bldg Type'], prefix='bldg_type', drop_first=True)

comb = pd.get_dummies(comb, columns=['House Style'], prefix='hse_style', drop_first=True)

comb = pd.get_dummies(comb, columns=['Roof Style'], prefix='rf_style', drop_first=True)

comb = pd.get_dummies(comb, columns=['Roof Matl'], prefix='rf_mat', drop_first=True)

comb = pd.get_dummies(comb, columns=['Exterior 1st'], prefix='ext1st', drop_first=True)

comb = pd.get_dummies(comb, columns=['Exterior 2nd'], prefix='ext2nd', drop_first=True)

comb = pd.get_dummies(comb, columns=['Mas Vnr Type'], prefix='mas_vnr_type', drop_first=True)

comb = pd.get_dummies(comb, columns=['Foundation'], prefix='found', drop_first=True)

comb = pd.get_dummies(comb, columns=['Heating'], prefix='heat', drop_first=True)

comb = pd.get_dummies(comb, columns=['Central Air'], prefix='ctrl_air', drop_first=True)

comb = pd.get_dummies(comb, columns=['Garage Type'], prefix='gar_type', drop_first=True)

comb = pd.get_dummies(comb, columns=['Misc Feature'], prefix='misc_feat', drop_first=True)

comb = pd.get_dummies(comb, columns=['Sale Type'], prefix='sale_type', drop_first=True)


In [520]:
comb.shape

(2816, 204)

__Ordinal encoding for `Exter Qual` column__

In [521]:
comb['Exter Qual'].value_counts()

TA    1702
Gd     983
Ex     107
Fa      24
Name: Exter Qual, dtype: int64

In [522]:
def conv_ordinal(x):
    if x == 'Po':
        return 0
    elif x == 'Fa':
        return 1 
    elif x == 'TA':
        return 2
    elif x == 'Gd':
        return 3
    elif x == 'Ex':
        return 4

comb['Exter Qual'] = comb['Exter Qual'].map(conv_ordinal)

In [523]:
comb['Exter Qual'].value_counts()

2    1702
3     983
4     107
1      24
Name: Exter Qual, dtype: int64

__Ordinal encoding for `Exter Cond` column__

In [524]:
comb['Exter Cond'].value_counts()

TA    2465
Gd     287
Fa      51
Ex      11
Po       2
Name: Exter Cond, dtype: int64

In [525]:
def conv_ordinal(x):
    if x == 'Po':
        return 0
    elif x == 'Fa':
        return 1 
    elif x == 'TA':
        return 2
    elif x == 'Gd':
        return 3
    elif x == 'Ex':
        return 4

comb['Exter Cond'] = comb['Exter Cond'].map(conv_ordinal)

In [526]:
comb['Exter Cond'].value_counts()

2    2465
3     287
1      51
4      11
0       2
Name: Exter Cond, dtype: int64

__Ordinal encoding for `Bsmt Qual` column__

In [527]:
comb['Bsmt Qual'].value_counts()

TA      1213
Gd      1194
Ex       258
Fa        78
None      71
Po         2
Name: Bsmt Qual, dtype: int64

In [528]:
def conv_ordinal(x):
    if x == 'None':
        return 0
    elif x == 'Po':
        return 1
    elif x == 'Fa':
        return 2 
    elif x == 'TA':
        return 3
    elif x == 'Gd':
        return 4
    elif x == 'Ex':
        return 5

comb['Bsmt Qual'] = comb['Bsmt Qual'].map(conv_ordinal)

In [529]:
comb['Bsmt Qual'].value_counts()

3    1213
4    1194
5     258
2      78
0      71
1       2
Name: Bsmt Qual, dtype: int64

__Ordinal encoding for `Bsmt Cond` column__

In [530]:
comb['Bsmt Cond'].value_counts()

TA      2531
Gd       117
Fa        92
None      71
Ex         3
Po         2
Name: Bsmt Cond, dtype: int64

In [531]:
def conv_ordinal(x):
    if x == 'None':
        return 0
    elif x == 'Po':
        return 1
    elif x == 'Fa':
        return 2 
    elif x == 'TA':
        return 3
    elif x == 'Gd':
        return 4
    elif x == 'Ex':
        return 5

comb['Bsmt Cond'] = comb['Bsmt Cond'].map(conv_ordinal)

In [532]:
comb['Bsmt Cond'].value_counts()

3    2531
4     117
2      92
0      71
5       3
1       2
Name: Bsmt Cond, dtype: int64

__Ordinal encoding for `Bsmt Exposure` column__

In [533]:
comb['Bsmt Exposure'].value_counts()

No      1826
Av       406
Gd       276
Mn       234
None      74
Name: Bsmt Exposure, dtype: int64

In [534]:
def conv_ordinal(x):
    if x == 'None':
        return 0
    elif x == 'No':
        return 1
    elif x == 'Mn':
        return 2 
    elif x == 'Av':
        return 3
    elif x == 'Gd':
        return 4

comb['Bsmt Exposure'] = comb['Bsmt Exposure'].map(conv_ordinal)

In [535]:
comb['Bsmt Exposure'].value_counts()

1    1826
3     406
4     276
2     234
0      74
Name: Bsmt Exposure, dtype: int64

__Ordinal encoding for `BsmtFin Type 1` column__

In [537]:
comb['BsmtFin Type 1'].value_counts()

GLQ     843
Unf     792
ALQ     415
Rec     282
BLQ     263
LwQ     150
None     71
Name: BsmtFin Type 1, dtype: int64

In [538]:
def conv_ordinal(x):
    if x == 'None':
        return 0
    elif x == 'Unf':
        return 1
    elif x == 'LwQ':
        return 2 
    elif x == 'Rec':
        return 3
    elif x == 'BLQ':
        return 4
    elif x == 'ALQ':
        return 5
    elif x == 'GLQ':
        return 6

comb['BsmtFin Type 1'] = comb['BsmtFin Type 1'].map(conv_ordinal)

In [539]:
comb['BsmtFin Type 1'].value_counts()

6    843
1    792
5    415
3    282
4    263
2    150
0     71
Name: BsmtFin Type 1, dtype: int64

__Ordinal encoding for `BsmtFin Type 2` column__

In [540]:
comb['BsmtFin Type 2'].value_counts()

Unf     2398
Rec      105
LwQ       89
None      72
BLQ       67
ALQ       52
GLQ       33
Name: BsmtFin Type 2, dtype: int64

In [541]:
def conv_ordinal(x):
    if x == 'None':
        return 0
    elif x == 'Unf':
        return 1
    elif x == 'LwQ':
        return 2 
    elif x == 'Rec':
        return 3
    elif x == 'BLQ':
        return 4
    elif x == 'ALQ':
        return 5
    elif x == 'GLQ':
        return 6

comb['BsmtFin Type 2'] = comb['BsmtFin Type 2'].map(conv_ordinal)

In [542]:
comb['BsmtFin Type 2'].value_counts()

1    2398
3     105
2      89
0      72
4      67
5      52
6      33
Name: BsmtFin Type 2, dtype: int64

__Ordinal encoding for `Heating QC` column__

In [543]:
comb['Heating QC'].value_counts()

Ex    1462
TA     818
Gd     455
Fa      78
Po       3
Name: Heating QC, dtype: int64

In [544]:
def conv_ordinal(x):
    if x == 'Po':
        return 0
    elif x == 'Fa':
        return 1
    elif x == 'TA':
        return 2 
    elif x == 'Gd':
        return 3
    elif x == 'Ex':
        return 4

comb['Heating QC'] = comb['Heating QC'].map(conv_ordinal)

In [545]:
comb['Heating QC'].value_counts()

4    1462
2     818
3     455
1      78
0       3
Name: Heating QC, dtype: int64

__Ordinal encoding for `Electrical` column__

In [546]:
comb['Electrical'].value_counts()

SBrkr    2596
FuseA     171
FuseF      43
FuseP       5
Mix         1
Name: Electrical, dtype: int64

In [547]:
def conv_ordinal(x):
    if x == 'Mix':
        return 0
    elif x == 'FuseP':
        return 1
    elif x == 'FuseF':
        return 2 
    elif x == 'FuseA':
        return 3
    elif x == 'SBrkr':
        return 4

comb['Electrical'] = comb['Electrical'].map(conv_ordinal)

In [548]:
comb['Electrical'].value_counts()

4    2596
3     171
2      43
1       5
0       1
Name: Electrical, dtype: int64

__Ordinal encoding for `Kitchen Qual` column__

In [549]:
comb['Kitchen Qual'].value_counts()

TA    1413
Gd    1145
Ex     204
Fa      53
Po       1
Name: Kitchen Qual, dtype: int64

In [550]:
def conv_ordinal(x):
    if x == 'Po':
        return 0
    elif x == 'Fa':
        return 1
    elif x == 'TA':
        return 2 
    elif x == 'Gd':
        return 3
    elif x == 'Ex':
        return 4

comb['Kitchen Qual'] = comb['Kitchen Qual'].map(conv_ordinal)

In [551]:
comb['Kitchen Qual'].value_counts()

2    1413
3    1145
4     204
1      53
0       1
Name: Kitchen Qual, dtype: int64

__Ordinal encoding for `Functional` column__

In [552]:
comb['Functional'].value_counts()

Typ     2626
Min2      67
Min1      64
Mod       32
Maj1      17
Maj2       7
Sal        2
Sev        1
Name: Functional, dtype: int64

In [553]:
def conv_ordinal(x):
    if x == 'Sal':
        return 0
    elif x == 'Sev':
        return 1
    elif x == 'Maj2':
        return 2 
    elif x == 'Maj1':
        return 3
    elif x == 'Mod':
        return 4
    elif x == 'Min2':
        return 5
    elif x == 'Min1':
        return 6
    elif x == 'Typ':
        return 7

comb['Functional'] = comb['Functional'].map(conv_ordinal)

In [554]:
comb['Functional'].value_counts()

7    2626
5      67
6      64
4      32
3      17
2       7
0       2
1       1
Name: Functional, dtype: int64

__Ordinal encoding for `Fireplace Qu` column__

In [555]:
comb['Fireplace Qu'].value_counts()

None    1317
Gd       738
TA       598
Fa        75
Po        45
Ex        43
Name: Fireplace Qu, dtype: int64

In [556]:
def conv_ordinal(x):
    if x == 'None':
        return 0
    elif x == 'Po':
        return 1
    elif x == 'Fa':
        return 2
    elif x == 'TA':
        return 3 
    elif x == 'Gd':
        return 4
    elif x == 'Ex':
        return 5

comb['Fireplace Qu'] = comb['Fireplace Qu'].map(conv_ordinal)

In [557]:
comb['Fireplace Qu'].value_counts()

0    1317
4     738
3     598
2      75
1      45
5      43
Name: Fireplace Qu, dtype: int64

__Ordinal encoding for `Garage Finish` column__

In [558]:
comb['Garage Finish'].value_counts()

Unf     1231
RFn      812
Fin      728
None      45
Name: Garage Finish, dtype: int64

In [559]:
def conv_ordinal(x):
    if x == 'None':
        return 0
    elif x == 'Unf':
        return 1
    elif x == 'RFn':
        return 2
    elif x == 'Fin':
        return 3 

comb['Garage Finish'] = comb['Garage Finish'].map(conv_ordinal)

In [560]:
comb['Garage Finish'].value_counts()

1    1231
2     812
3     728
0      45
Name: Garage Finish, dtype: int64

__Ordinal encoding for `Garage Qual` column__

In [561]:
comb['Garage Qual'].value_counts()

TA      2615
Fa       124
None      45
Gd        24
Po         5
Ex         3
Name: Garage Qual, dtype: int64

In [562]:
def conv_ordinal(x):
    if x == 'None':
        return 0
    elif x == 'Po':
        return 1
    elif x == 'Fa':
        return 2
    elif x == 'TA':
        return 3 
    elif x == 'Gd':
        return 4
    elif x == 'Ex':
        return 5

comb['Garage Qual'] = comb['Garage Qual'].map(conv_ordinal)

In [563]:
comb['Garage Qual'].value_counts()

3    2615
2     124
0      45
4      24
1       5
5       3
Name: Garage Qual, dtype: int64

__Ordinal encoding for `Garage Cond` column__

In [564]:
comb['Garage Cond'].value_counts()

TA      2665
Fa        74
None      45
Gd        15
Po        14
Ex         3
Name: Garage Cond, dtype: int64

In [565]:
def conv_ordinal(x):
    if x == 'None':
        return 0
    elif x == 'Po':
        return 1
    elif x == 'Fa':
        return 2
    elif x == 'TA':
        return 3 
    elif x == 'Gd':
        return 4
    elif x == 'Ex':
        return 5

comb['Garage Cond'] = comb['Garage Cond'].map(conv_ordinal)

In [566]:
comb['Garage Cond'].value_counts()

3    2665
2      74
0      45
4      15
1      14
5       3
Name: Garage Cond, dtype: int64

__Ordinal encoding for `Paved Drive` column__

In [567]:
comb['Paved Drive'].value_counts()

Y    2593
N     164
P      59
Name: Paved Drive, dtype: int64

In [568]:
def conv_ordinal(x):
    if x == 'N':
        return 0
    elif x == 'P':
        return 1
    elif x == 'Y':
        return 2

comb['Paved Drive'] = comb['Paved Drive'].map(conv_ordinal)

In [569]:
comb['Paved Drive'].value_counts()

2    2593
0     164
1      59
Name: Paved Drive, dtype: int64

__Ordinal encoding for `Pool QC` column__

In [570]:
comb['Pool QC'].value_counts()

None    2803
Ex         4
Gd         4
TA         3
Fa         2
Name: Pool QC, dtype: int64

In [571]:
def conv_ordinal(x):
    if x == 'None':
        return 0
    elif x == 'Fa':
        return 1
    elif x == 'TA':
        return 2 
    elif x == 'Gd':
        return 3
    elif x == 'Ex':
        return 4

comb['Pool QC'] = comb['Pool QC'].map(conv_ordinal)

In [572]:
comb['Pool QC'].value_counts()

0    2803
3       4
4       4
2       3
1       2
Name: Pool QC, dtype: int64

__Ordinal encoding for `Fence` column__

In [573]:
comb['Fence'].value_counts()

None     2265
MnPrv     317
GdPrv     117
GdWo      105
MnWw       12
Name: Fence, dtype: int64

In [574]:
def conv_ordinal(x):
    if x == 'None':
        return 0
    elif x == 'MnWw':
        return 1
    elif x == 'GdWo':
        return 2 
    elif x == 'MnPrv':
        return 3
    elif x == 'GdPrv':
        return 4

comb['Fence'] = comb['Fence'].map(conv_ordinal)

In [575]:
comb['Fence'].value_counts()

0    2265
3     317
4     117
2     105
1      12
Name: Fence, dtype: int64

In [576]:
#ensure no more categorical data
comb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2816 entries, 0 to 2815
Columns: 204 entries, Id to sale_type_WD 
dtypes: float64(11), int64(48), uint8(145)
memory usage: 1.7 MB


In [578]:
#check for float values to ensure only continuous variables should be float
[i for i in comb.columns if comb[i].dtype == float]

['Lot Frontage',
 'Mas Vnr Area',
 'BsmtFin SF 1',
 'BsmtFin SF 2',
 'Bsmt Unf SF',
 'Total Bsmt SF',
 'Bsmt Full Bath',
 'Bsmt Half Bath',
 'Garage Yr Blt',
 'Garage Cars',
 'Garage Area']

From here we can see that `Garage Yr Blt` which means the year that the garage was being built is in float. Ideally, this should be a int.

In [582]:
comb['Garage Yr Blt'].head()

0    1976.0
1    1997.0
2    1953.0
3    2007.0
4    1957.0
Name: Garage Yr Blt, dtype: float64

__Changing of `Garage Yr Blt` data type__

In [586]:
comb['Garage Yr Blt'] = comb['Garage Yr Blt'].astype('int64') 

In [587]:
comb['Garage Yr Blt'].dtype

dtype('int64')

In [588]:
comb['Garage Yr Blt'].head()

0    1976
1    1997
2    1953
3    2007
4    1957
Name: Garage Yr Blt, dtype: int64

In [590]:
comb.shape

(2816, 204)

### Save clean combined encoded dataset

In [592]:
#save it to csv
comb.to_csv('./datasets/clean_comb_encoded.csv', index=False)

In [593]:
print(train_df.shape)
print(test_df.shape)

(1937, 79)
(879, 79)


,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
1932,1587,921126030,20,RL,79.0,11449,Pave,IR1,HLS,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,1Story,8,5,2007,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,Av,GLQ,1011.0,Unf,0.0,873.0,1884.0,GasA,Ex,Y,SBrkr,1728,0,0,1728,1.0,0.0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2007.0,Fin,2.0,520.0,TA,TA,Y,0,276,0,0,0,0,None,None,None,0,1,2008,WD
1933,785,905377130,30,RL,0.0,12342,Pave,IR1,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,4,5,1940,1950,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,262.0,Unf,0.0,599.0,861.0,GasA,Ex,Y,SBrkr,861,0,0,861,0.0,0.0,1,0,1,1,TA,4,Typ,0,None,Detchd,1961.0,Unf,2.0,539.0,TA,TA,Y,158,0,0,0,0,0,None,None,None,0,3,2009,WD
1934,916,909253010,50,RL,57.0,7558,Pave,Reg,Bnk,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,1.5Fin,6,6,1928,1950,Gable,CompShg,BrkFace,Stone,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,896.0,896.0,GasA,Gd,Y,SBrkr,1172,741,0,1913,0.0,0.0,1,1,3,1,TA,9,Typ,1,TA,Detchd,1929.0,Unf,2.0,342.0,Fa,Fa,Y,0,0,0,0,0,0,None,None,None,0,3,2009,WD
1935,639,535179160,20,RL,80.0,10400,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,4,5,1956,1956,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,155.0,LwQ,750.0,295.0,1200.0,GasA,TA,Y,SBrkr,1200,0,0,1200,1.0,0.0,1,0,3,1,TA,6,Typ,2,Gd,Attchd,1956.0,Unf,1.0,294.0,TA,TA,Y,0,189,140,0,0,0,None,None,None,0,11,2009,WD
1936,10,527162130,60,RL,60.0,7500,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,1999,1999,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,TA,TA,No,Unf,0.0,Unf,0.0,994.0,994.0,GasA,Gd,Y,SBrkr,1028,776,0,1804,0.0,0.0,2,1,3,1,Gd,7,Typ,1,TA,Attchd,1999.0,Fin,2.0,442.0,TA,TA,Y,140,60,0,0,0,0,None,None,None,0,6,2010,WD


### Split back into train and test set

In [600]:
#split to train set
train_enc = comb.iloc[:1937, :]
train_enc.shape

(1937, 204)

In [610]:
#split to test set 
test_enc = comb.iloc[1937:, :]
print(test_enc.shape)
test_enc.reset_index(drop=True, inplace=True)

(879, 204)


In [613]:
test_enc.head()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Lot Shape,Utilities,Land Slope,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,Exter Qual,Exter Cond,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating QC,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Val,Mo Sold,Yr Sold,zoning_C,zoning_FV,zoning_I,zoning_RH,zoning_RL,zoning_RM,street_Pave,land_countour_HLS,land_countour_Low,land_countour_Lvl,lot_conf_CulDSac,lot_conf_FR2,lot_conf_FR3,lot_conf_Inside,neigh_Blueste,neigh_BrDale,neigh_BrkSide,neigh_ClearCr,neigh_CollgCr,neigh_Crawfor,neigh_Edwards,neigh_Gilbert,neigh_Greens,neigh_GrnHill,neigh_IDOTRR,neigh_Landmrk,neigh_MeadowV,neigh_Mitchel,neigh_NAmes,neigh_NPkVill,neigh_NWAmes,neigh_NoRidge,neigh_NridgHt,neigh_OldTown,neigh_SWISU,neigh_Sawyer,neigh_SawyerW,neigh_Somerst,neigh_StoneBr,neigh_Timber,neigh_Veenker,cond1_Feedr,cond1_Norm,cond1_PosA,cond1_PosN,cond1_RRAe,cond1_RRAn,cond1_RRNe,cond1_RRNn,cond2_Feedr,cond2_Norm,cond2_PosA,cond2_PosN,cond2_RRAe,cond2_RRAn,cond2_RRNn,bldg_type_2fmCon,bldg_type_Duplex,bldg_type_Twnhs,bldg_type_TwnhsE,hse_style_1.5Unf,hse_style_1Story,hse_style_2.5Fin,hse_style_2.5Unf,hse_style_2Story,hse_style_SFoyer,hse_style_SLvl,rf_style_Gable,rf_style_Gambrel,rf_style_Hip,rf_style_Mansard,rf_style_Shed,rf_mat_CompShg,rf_mat_Membran,rf_mat_Metal,rf_mat_Roll,rf_mat_Tar&Grv,rf_mat_WdShake,rf_mat_WdShngl,ext1st_AsphShn,ext1st_BrkComm,ext1st_BrkFace,ext1st_CBlock,ext1st_CemntBd,ext1st_HdBoard,ext1st_ImStucc,ext1st_MetalSd,ext1st_Plywood,ext1st_PreCast,ext1st_Stone,ext1st_Stucco,ext1st_VinylSd,ext1st_Wd Sdng,ext1st_WdShing,ext2nd_AsphShn,ext2nd_Brk Cmn,ext2nd_BrkFace,ext2nd_CBlock,ext2nd_CmentBd,ext2nd_HdBoard,ext2nd_ImStucc,ext2nd_MetalSd,ext2nd_Other,ext2nd_Plywood,ext2nd_PreCast,ext2nd_Stone,ext2nd_Stucco,ext2nd_VinylSd,ext2nd_Wd Sdng,ext2nd_Wd Shng,mas_vnr_type_BrkFace,mas_vnr_type_CBlock,mas_vnr_type_None,mas_vnr_type_Stone,found_CBlock,found_PConc,found_Slab,found_Stone,found_Wood,heat_GasA,heat_GasW,heat_Grav,heat_OthW,heat_Wall,ctrl_air_Y,gar_type_Attchd,gar_type_Basment,gar_type_BuiltIn,gar_type_CarPort,gar_type_Detchd,gar_type_None,misc_feat_Gar2,misc_feat_None,misc_feat_Othr,misc_feat_Shed,misc_feat_TenC,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_VWD,sale_type_WD
0,2658,902301120,190,69.0,9142,3,3,2,6,8,1910,1950,0.0,2,1,2,3,1,1,0.0,1,0.0,1020.0,1020.0,3,1,908,1020,0,1928,0.0,0.0,2,0,4,2,1,9,7,0,0,1910,1,1.0,440.0,1,1,2,0,60,112,0,0,0,0,0,0,4,2006,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,2718,905108090,90,0.0,9662,2,3,2,5,4,1977,1977,0.0,2,2,4,3,1,1,0.0,1,0.0,1967.0,1967.0,2,4,1967,0,0,1967,0.0,0.0,2,0,6,2,2,10,7,0,0,1977,3,2.0,580.0,3,3,2,170,0,0,0,0,0,0,0,0,8,2006,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,2414,528218130,60,58.0,17104,2,3,2,7,5,2006,2006,0.0,3,2,4,4,3,6,554.0,1,0.0,100.0,654.0,4,4,664,832,0,1496,1.0,0.0,2,1,3,1,3,7,7,1,4,2006,2,2.0,426.0,3,3,2,100,24,0,0,0,0,0,0,0,9,2006,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,

In [609]:
test_df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2Story,6,8,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,Fa,Stone,Fa,TA,No,Unf,0,Unf,0,1020,1020,GasA,Gd,N,FuseP,908,1020,0,1928,0,0,2,0,4,2,Fa,9,Typ,0,None,Detchd,1910.0,Unf,1,440,Po,Po,Y,0,60,112,0,0,0,None,None,None,0,4,2006,WD
1,2718,905108090,90,RL,0.0,9662,Pave,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,Duplex,1Story,5,4,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,Gd,TA,No,Unf,0,Unf,0,1967,1967,GasA,TA,Y,SBrkr,1967,0,0,1967,0,0,2,0,6,2,TA,10,Typ,0,None,Attchd,1977.0,Fin,2,580,TA,TA,Y,170,0,0,0,0,0,None,None,None,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,Gd,Av,GLQ,554,Unf,0,100,654,GasA,Ex,Y,SBrkr,664,832,0,1496,1,0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,2006.0,RFn,2,426,TA,TA,Y,100,24,0,0,0,0,None,None,None,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Gd,TA,CBlock,TA,TA,No,Unf,0,Unf,0,968,968,GasA,TA,Y,SBrkr,968,0,0,968,0,0,1,0,2,1,TA,5,Typ,0,None,Detchd,1935.0,Unf,2,480,Fa,TA,N,0,0,184,0,0,0,None,None,None,0,7,2007,WD
4,625,535105100,20,RL,0.0,9500,Pave,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,TA,TA,CBlock,Gd,TA,No,BLQ,609,Unf,0,785,1394,GasA,Gd,Y,SBrkr,1394,0,0,1394,1,0,1,1,3,1,TA,6,Typ,2,Gd,Attchd,1963.0,RFn,2,514,TA,TA,Y,0,76,0,0,185,0,None,None,None,0,7,2009,WD


In [605]:
train_enc.tail()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Lot Shape,Utilities,Land Slope,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,Exter Qual,Exter Cond,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating QC,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Val,Mo Sold,Yr Sold,zoning_C,zoning_FV,zoning_I,zoning_RH,zoning_RL,zoning_RM,street_Pave,land_countour_HLS,land_countour_Low,land_countour_Lvl,lot_conf_CulDSac,lot_conf_FR2,lot_conf_FR3,lot_conf_Inside,neigh_Blueste,neigh_BrDale,neigh_BrkSide,neigh_ClearCr,neigh_CollgCr,neigh_Crawfor,neigh_Edwards,neigh_Gilbert,neigh_Greens,neigh_GrnHill,neigh_IDOTRR,neigh_Landmrk,neigh_MeadowV,neigh_Mitchel,neigh_NAmes,neigh_NPkVill,neigh_NWAmes,neigh_NoRidge,neigh_NridgHt,neigh_OldTown,neigh_SWISU,neigh_Sawyer,neigh_SawyerW,neigh_Somerst,neigh_StoneBr,neigh_Timber,neigh_Veenker,cond1_Feedr,cond1_Norm,cond1_PosA,cond1_PosN,cond1_RRAe,cond1_RRAn,cond1_RRNe,cond1_RRNn,cond2_Feedr,cond2_Norm,cond2_PosA,cond2_PosN,cond2_RRAe,cond2_RRAn,cond2_RRNn,bldg_type_2fmCon,bldg_type_Duplex,bldg_type_Twnhs,bldg_type_TwnhsE,hse_style_1.5Unf,hse_style_1Story,hse_style_2.5Fin,hse_style_2.5Unf,hse_style_2Story,hse_style_SFoyer,hse_style_SLvl,rf_style_Gable,rf_style_Gambrel,rf_style_Hip,rf_style_Mansard,rf_style_Shed,rf_mat_CompShg,rf_mat_Membran,rf_mat_Metal,rf_mat_Roll,rf_mat_Tar&Grv,rf_mat_WdShake,rf_mat_WdShngl,ext1st_AsphShn,ext1st_BrkComm,ext1st_BrkFace,ext1st_CBlock,ext1st_CemntBd,ext1st_HdBoard,ext1st_ImStucc,ext1st_MetalSd,ext1st_Plywood,ext1st_PreCast,ext1st_Stone,ext1st_Stucco,ext1st_VinylSd,ext1st_Wd Sdng,ext1st_WdShing,ext2nd_AsphShn,ext2nd_Brk Cmn,ext2nd_BrkFace,ext2nd_CBlock,ext2nd_CmentBd,ext2nd_HdBoard,ext2nd_ImStucc,ext2nd_MetalSd,ext2nd_Other,ext2nd_Plywood,ext2nd_PreCast,ext2nd_Stone,ext2nd_Stucco,ext2nd_VinylSd,ext2nd_Wd Sdng,ext2nd_Wd Shng,mas_vnr_type_BrkFace,mas_vnr_type_CBlock,mas_vnr_type_None,mas_vnr_type_Stone,found_CBlock,found_PConc,found_Slab,found_Stone,found_Wood,heat_GasA,heat_GasW,heat_Grav,heat_OthW,heat_Wall,ctrl_air_Y,gar_type_Attchd,gar_type_Basment,gar_type_BuiltIn,gar_type_CarPort,gar_type_Detchd,gar_type_None,misc_feat_Gar2,misc_feat_None,misc_feat_Othr,misc_feat_Shed,misc_feat_TenC,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_VWD,sale_type_WD
1932,1587,921126030,20,79.0,11449,2,3,2,8,5,2007,2007,0.0,3,2,4,3,3,6,1011.0,1,0.0,873.0,1884.0,4,4,1728,0,0,1728,1.0,0.0,2,0,3,1,3,7,7,1,4,2007,3,2.0,520.0,3,3,2,0,276,0,0,0,0,0,0,0,1,2008,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1933,785,905377130,30,0.0,12342,2,3,2,4,5,1940,1950,0.0,2,2,3,3,1,4,262.0,1,0.0,599.0,861.0,4,4,861,0,0,861,0.0,0.0,1,0,1,1,2,4,7,0,0,1961,1,2.0,539.0,3,3,2,158,0,0,0,0,0,0,0,0,3,2009,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1934,916,909253010,50,57.0,7558,3,3,2,6,6,1928,1950,0.0,2,2,3,3,1,1,0.0,1,0.0,896.0,896.0,3,4,1172,741,0,1913,0.0,0.0,1,1,3,1,2,9,7,1,3,1929,1,2.0,342.0,2,2,2,0,0,0,0,0,0,0,0,0,3,2009,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0

In [606]:
train_df.tail()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
1932,1587,921126030,20,RL,79.0,11449,Pave,IR1,HLS,AllPub,Inside,Gtl,Timber,Norm,Norm,1Fam,1Story,8,5,2007,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,Av,GLQ,1011.0,Unf,0.0,873.0,1884.0,GasA,Ex,Y,SBrkr,1728,0,0,1728,1.0,0.0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2007.0,Fin,2.0,520.0,TA,TA,Y,0,276,0,0,0,0,None,None,None,0,1,2008,WD
1933,785,905377130,30,RL,0.0,12342,Pave,IR1,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,4,5,1940,1950,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,262.0,Unf,0.0,599.0,861.0,GasA,Ex,Y,SBrkr,861,0,0,861,0.0,0.0,1,0,1,1,TA,4,Typ,0,None,Detchd,1961.0,Unf,2.0,539.0,TA,TA,Y,158,0,0,0,0,0,None,None,None,0,3,2009,WD
1934,916,909253010,50,RL,57.0,7558,Pave,Reg,Bnk,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,1.5Fin,6,6,1928,1950,Gable,CompShg,BrkFace,Stone,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0.0,Unf,0.0,896.0,896.0,GasA,Gd,Y,SBrkr,1172,741,0,1913,0.0,0.0,1,1,3,1,TA,9,Typ,1,TA,Detchd,1929.0,Unf,2.0,342.0,Fa,Fa,Y,0,0,0,0,0,0,None,None,None,0,3,2009,WD
1935,639,535179160,20,RL,80.0,10400,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,4,5,1956,1956,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,155.0,LwQ,750.0,295.0,1200.0,GasA,TA,Y,SBrkr,1200,0,0,1200,1.0,0.0,1,0,3,1,TA,6,Typ,2,Gd,Attchd,1956.0,Unf,1.0,294.0,TA,TA,Y,0,189,140,0,0,0,None,None,None,0,11,2009,WD
1936,10,527162130,60,RL,60.0,7500,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,1999,1999,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,TA,TA,No,Unf,0.0,Unf,0.0,994.0,994.0,GasA,Gd,Y,SBrkr,1028,776,0,1804,0.0,0.0,2,1,3,1,Gd,7,Typ,1,TA,Attchd,1999.0,Fin,2.0,442.0,TA,TA,Y,140,60,0,0,0,0,None,None,None,0,6,2010,WD


In [615]:
train_sale_price.head()

,SalePrice
0,130500
1,220000
2,109000
3,174000
4,138500


In [617]:
train_enc.head()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Lot Shape,Utilities,Land Slope,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,Exter Qual,Exter Cond,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating QC,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Val,Mo Sold,Yr Sold,zoning_C,zoning_FV,zoning_I,zoning_RH,zoning_RL,zoning_RM,street_Pave,land_countour_HLS,land_countour_Low,land_countour_Lvl,lot_conf_CulDSac,lot_conf_FR2,lot_conf_FR3,lot_conf_Inside,neigh_Blueste,neigh_BrDale,neigh_BrkSide,neigh_ClearCr,neigh_CollgCr,neigh_Crawfor,neigh_Edwards,neigh_Gilbert,neigh_Greens,neigh_GrnHill,neigh_IDOTRR,neigh_Landmrk,neigh_MeadowV,neigh_Mitchel,neigh_NAmes,neigh_NPkVill,neigh_NWAmes,neigh_NoRidge,neigh_NridgHt,neigh_OldTown,neigh_SWISU,neigh_Sawyer,neigh_SawyerW,neigh_Somerst,neigh_StoneBr,neigh_Timber,neigh_Veenker,cond1_Feedr,cond1_Norm,cond1_PosA,cond1_PosN,cond1_RRAe,cond1_RRAn,cond1_RRNe,cond1_RRNn,cond2_Feedr,cond2_Norm,cond2_PosA,cond2_PosN,cond2_RRAe,cond2_RRAn,cond2_RRNn,bldg_type_2fmCon,bldg_type_Duplex,bldg_type_Twnhs,bldg_type_TwnhsE,hse_style_1.5Unf,hse_style_1Story,hse_style_2.5Fin,hse_style_2.5Unf,hse_style_2Story,hse_style_SFoyer,hse_style_SLvl,rf_style_Gable,rf_style_Gambrel,rf_style_Hip,rf_style_Mansard,rf_style_Shed,rf_mat_CompShg,rf_mat_Membran,rf_mat_Metal,rf_mat_Roll,rf_mat_Tar&Grv,rf_mat_WdShake,rf_mat_WdShngl,ext1st_AsphShn,ext1st_BrkComm,ext1st_BrkFace,ext1st_CBlock,ext1st_CemntBd,ext1st_HdBoard,ext1st_ImStucc,ext1st_MetalSd,ext1st_Plywood,ext1st_PreCast,ext1st_Stone,ext1st_Stucco,ext1st_VinylSd,ext1st_Wd Sdng,ext1st_WdShing,ext2nd_AsphShn,ext2nd_Brk Cmn,ext2nd_BrkFace,ext2nd_CBlock,ext2nd_CmentBd,ext2nd_HdBoard,ext2nd_ImStucc,ext2nd_MetalSd,ext2nd_Other,ext2nd_Plywood,ext2nd_PreCast,ext2nd_Stone,ext2nd_Stucco,ext2nd_VinylSd,ext2nd_Wd Sdng,ext2nd_Wd Shng,mas_vnr_type_BrkFace,mas_vnr_type_CBlock,mas_vnr_type_None,mas_vnr_type_Stone,found_CBlock,found_PConc,found_Slab,found_Stone,found_Wood,heat_GasA,heat_GasW,heat_Grav,heat_OthW,heat_Wall,ctrl_air_Y,gar_type_Attchd,gar_type_Basment,gar_type_BuiltIn,gar_type_CarPort,gar_type_Detchd,gar_type_None,misc_feat_Gar2,misc_feat_None,misc_feat_Othr,misc_feat_Shed,misc_feat_TenC,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_VWD,sale_type_WD
0,109,533352170,60,0.0,13517,2,3,2,6,8,1976,2005,289.0,3,2,3,3,1,6,533.0,1,0.0,192.0,725.0,4,4,725,754,0,1479,0.0,0.0,2,1,3,1,3,6,7,0,0,1976,2,2.0,475.0,3,3,2,0,44,0,0,0,0,0,0,0,3,2010,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,544,531379050,60,43.0,11492,2,3,2,7,5,1996,1997,132.0,3,2,4,3,1,6,637.0,1,0.0,276.0,913.0,4,4,913,1209,0,2122,1.0,0.0,2,1,4,1,3,8,7,1,3,1997,2,2.0,559.0,3,3,2,0,74,0,0,0,0,0,0,0,4,2009,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,153,535304180,20,68.0,7922,3,3,2,5,7,1953,2007,0.0,2,3,3,3,1,6,731.0,1,0.0,326.0,1057.0,2,4,1057,0,0,1057,1.0,0.0,1,0,3,1,3,5,7,0,0,1953,1,1.0,246.0,3,3,2,0,52,0,0,0,0,0,0,0,1,2010,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,

In [625]:
#patch back sale price in training set
train_enc = pd.merge(train_enc, train_sale_price, how='inner', left_index=True, right_index=True)

In [626]:
train_enc.head()

,Id,PID,MS SubClass,Lot Frontage,Lot Area,Lot Shape,Utilities,Land Slope,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,Exter Qual,Exter Cond,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating QC,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Val,Mo Sold,Yr Sold,zoning_C,zoning_FV,zoning_I,zoning_RH,zoning_RL,zoning_RM,street_Pave,land_countour_HLS,land_countour_Low,land_countour_Lvl,lot_conf_CulDSac,lot_conf_FR2,lot_conf_FR3,lot_conf_Inside,neigh_Blueste,neigh_BrDale,neigh_BrkSide,neigh_ClearCr,neigh_CollgCr,neigh_Crawfor,neigh_Edwards,neigh_Gilbert,neigh_Greens,neigh_GrnHill,neigh_IDOTRR,neigh_Landmrk,neigh_MeadowV,neigh_Mitchel,neigh_NAmes,neigh_NPkVill,neigh_NWAmes,neigh_NoRidge,neigh_NridgHt,neigh_OldTown,neigh_SWISU,neigh_Sawyer,neigh_SawyerW,neigh_Somerst,neigh_StoneBr,neigh_Timber,neigh_Veenker,cond1_Feedr,cond1_Norm,cond1_PosA,cond1_PosN,cond1_RRAe,cond1_RRAn,cond1_RRNe,cond1_RRNn,cond2_Feedr,cond2_Norm,cond2_PosA,cond2_PosN,cond2_RRAe,cond2_RRAn,cond2_RRNn,bldg_type_2fmCon,bldg_type_Duplex,bldg_type_Twnhs,bldg_type_TwnhsE,hse_style_1.5Unf,hse_style_1Story,hse_style_2.5Fin,hse_style_2.5Unf,hse_style_2Story,hse_style_SFoyer,hse_style_SLvl,rf_style_Gable,rf_style_Gambrel,rf_style_Hip,rf_style_Mansard,rf_style_Shed,rf_mat_CompShg,rf_mat_Membran,rf_mat_Metal,rf_mat_Roll,rf_mat_Tar&Grv,rf_mat_WdShake,rf_mat_WdShngl,ext1st_AsphShn,ext1st_BrkComm,ext1st_BrkFace,ext1st_CBlock,ext1st_CemntBd,ext1st_HdBoard,ext1st_ImStucc,ext1st_MetalSd,ext1st_Plywood,ext1st_PreCast,ext1st_Stone,ext1st_Stucco,ext1st_VinylSd,ext1st_Wd Sdng,ext1st_WdShing,ext2nd_AsphShn,ext2nd_Brk Cmn,ext2nd_BrkFace,ext2nd_CBlock,ext2nd_CmentBd,ext2nd_HdBoard,ext2nd_ImStucc,ext2nd_MetalSd,ext2nd_Other,ext2nd_Plywood,ext2nd_PreCast,ext2nd_Stone,ext2nd_Stucco,ext2nd_VinylSd,ext2nd_Wd Sdng,ext2nd_Wd Shng,mas_vnr_type_BrkFace,mas_vnr_type_CBlock,mas_vnr_type_None,mas_vnr_type_Stone,found_CBlock,found_PConc,found_Slab,found_Stone,found_Wood,heat_GasA,heat_GasW,heat_Grav,heat_OthW,heat_Wall,ctrl_air_Y,gar_type_Attchd,gar_type_Basment,gar_type_BuiltIn,gar_type_CarPort,gar_type_Detchd,gar_type_None,misc_feat_Gar2,misc_feat_None,misc_feat_Othr,misc_feat_Shed,misc_feat_TenC,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_VWD,sale_type_WD,SalePrice
0,109,533352170,60,0.0,13517,2,3,2,6,8,1976,2005,289.0,3,2,3,3,1,6,533.0,1,0.0,192.0,725.0,4,4,725,754,0,1479,0.0,0.0,2,1,3,1,3,6,7,0,0,1976,2,2.0,475.0,3,3,2,0,44,0,0,0,0,0,0,0,3,2010,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,130500
1,544,531379050,60,43.0,11492,2,3,2,7,5,1996,1997,132.0,3,2,4,3,1,6,637.0,1,0.0,276.0,913.0,4,4,913,1209,0,2122,1.0,0.0,2,1,4,1,3,8,7,1,3,1997,2,2.0,559.0,3,3,2,0,74,0,0,0,0,0,0,0,4,2009,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,220000
2,153,535304180,20,68.0,7922,3,3,2,5,7,1953,2007,0.0,2,3,3,3,1,6,731.0,1,0.0,326.0,1057.0,2,4,1057,0,0,1057,1.0,0.0,1,0,3,1,3,5,7,0,0,1953,1,1.0,246.0,3,3,2,0,52,0,0,0,0,0,0,0,1,2010,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,

In [628]:
print(train_enc.shape)
test_enc.shape

(1937, 205)


(879, 204)

In [627]:
#save it to csv
train_enc.to_csv('./datasets/clean_train_encoded.csv', index=False)
test_enc.to_csv('./datasets/clean_test_encoded.csv', index=False)